In [1]:
import pandas as pd 
import os 
import numpy as np

import time
import re

import sys
import gc 


In [2]:
# Intel only
# !pip install scikit-learn-intelex
from sklearnex import patch_sklearn
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
os.getcwd()

'/home/quanghung20gg/Documents/new/datavis_final/datavis_final/eda/v1'

In [5]:
np.random.seed(0)

In [6]:
UPSAMPLE_RATIO = 1.5

In [7]:
ROOT = '/kaggle/input/dseb-64-data-preparation-final-project/dseb63_final_project_DP_dataset/dseb63_final_project_DP_dataset'
ROOT = '../../data'

# Preprocessing

## APP PREV

In [8]:
def process_app_prev():
    prev_app = pd.read_csv(os.path.join(ROOT, "dseb63_previous_application.csv"))
    prev_app['DATE_DIFF'] = prev_app['DAYS_TERMINATION'] - prev_app['DAYS_LAST_DUE']
    good_replace = ['Gardening', 'Animals', 'Insurance', 'Medicine', 'Fitness', 'Direct Sales', 'Additional Service', 'Education', 'Weapon', ]
    good_replace_2 = ['Vehicles','Homewares','Medical Supplies','Jewelry', 'Tourism', 'Office Appliances']
    # good_replace+=good_replace_2
    
    prev_app['NAME_GOODS_CATEGORY'] = prev_app['NAME_GOODS_CATEGORY'].replace(good_replace, 'Other')
    prev_app['NAME_CASH_LOAN_PURPOSE'].replace({'Wedding / gift / holiday':'Journey', 'Hobby': 'Everyday expenses'}, inplace=True)
    
    loan_purpose = ['Refusal to name the goal', 'Money for a third person', 'Buying a garage', 'Buying a garage', 'Gasification / water supply', 'Business development', 'Buying a holiday home / land', 'Furniture', 'Car repairs', 'Buying a home',  'Purchase of electronic equipment', 'Journey', 'Education', 'Payments on other loans', 'Medicine', 'Building a house or an annex']
    
    prev_app['NAME_CASH_LOAN_PURPOSE'] = prev_app['NAME_CASH_LOAN_PURPOSE'].replace(loan_purpose, 'Other')
    prev_app['NAME_CASH_LOAN_PURPOSE'] = prev_app['NAME_CASH_LOAN_PURPOSE'].replace(['Buying a new car', 'Buying a used car'], 'Buying a car')
    prev_app['FLAG_LAST_APPL_PER_CONTRACT'] = prev_app['FLAG_LAST_APPL_PER_CONTRACT'].map({'Y': 1, 'N': 0}).astype(int)
    prev_app['NAME_TYPE_SUITE'].replace('Children', 'Family', inplace=True)
    prev_app['NAME_TYPE_SUITE'].replace('Other_A', 'Other', inplace=True)
    prev_app['NAME_TYPE_SUITE'].replace('Other_B', 'Other', inplace=True)
    prev_app['NAME_TYPE_SUITE'].replace('Group of people', 'Other', inplace=True)
    prev_app['NAME_TYPE_SUITE'].replace('Spouse, partner', 'Family', inplace=True)
    prev_app['NAME_TYPE_SUITE'].replace('Unaccompanied', 'Single', inplace=True)

    prev_app.replace({'NAME_YIELD_GROUP': {'XNA': 0, 'low_action': 1, 'low_normal': 1, 'middle': 3, 'high': 4}}, inplace=True)
    prev_app['NAME_YIELD_GROUP'].fillna(0, inplace=True)

    def OHE(df, nan_as_category = True):
        original_cols = list(df.columns)
        categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
        # for col in categorical_columns:
        #     df[col].fillna('XNA', inplace=True)

        df = pd.get_dummies(df, columns= categorical_columns,
                            dummy_na= nan_as_category)
        new_columns = [c for c in df.columns if c not in original_cols]
        return df, new_columns

    ## Hung feat
    prev_app['ACTIVE'] = prev_app['DAYS_LAST_DUE'] > 0
    prev_app['ACTUAL_DURATION'] = prev_app['DAYS_LAST_DUE'] - prev_app['DAYS_FIRST_DUE']
    prev_app['DURATION'] = prev_app['DAYS_TERMINATION'] - prev_app['DAYS_FIRST_DUE']
    prev_app['LIFETIME_LOAN'] = prev_app['DAYS_TERMINATION'] > 16000

    prev_app['FINISH_RATE'] = (prev_app['ACTUAL_DURATION'] / prev_app['DURATION']) * (1 - prev_app['ACTIVE']) * (1 - prev_app['LIFETIME_LOAN'])

    # TEST
    mask_duration = (prev_app['DURATION'] == 0)
    prev_app.loc[mask_duration, 'FINISH_RATE'] = np.nan

    #New features from old features

    #Ratio: value ask / value received percentage 
    prev_app = prev_app.drop(prev_app[prev_app['AMT_CREDIT'].isnull() == True].index) 
    prev_app['AMT_DOWN_PAYMENT'].fillna(0, inplace=True)
    prev_app['APP_CREDIT_PERC'] = prev_app['AMT_APPLICATION'] / prev_app['AMT_CREDIT'] * (prev_app['AMT_CREDIT'] != 0)

    #dgp
    prev_app['DOWN_PAYMENT_RATIO'] = prev_app['AMT_DOWN_PAYMENT']/ prev_app['AMT_CREDIT'] * (prev_app['AMT_CREDIT'] != 0)
    prev_app['GOODS_PRICE_RATIO'] = prev_app['AMT_GOODS_PRICE'] / prev_app['AMT_CREDIT'] * (prev_app['AMT_CREDIT'] != 0)


    # v1.2
    mask_AMT_CREDIT = (prev_app['AMT_CREDIT'] == 0) 
    prev_app.loc[mask_AMT_CREDIT, 'APP_CREDIT_PERC'] = np.nan
    prev_app.loc[mask_AMT_CREDIT, 'DOWN_PAYMENT_RATIO'] = np.nan 
    prev_app.loc[mask_AMT_CREDIT, 'GOODS_PRICE_RATIO'] = np.nan

    # insurance
    prev_app['Insuranced_amt_application'] = prev_app['AMT_APPLICATION'] * prev_app['NFLAG_INSURED_ON_APPROVAL']
    prev_app['Insuranced_amt_credit'] = prev_app['AMT_CREDIT'] * prev_app['NFLAG_INSURED_ON_APPROVAL']

    prev_app['HIGH_AMT_APPLICATION'] = (prev_app['AMT_APPLICATION'] > 100000)
    prev_app['Insuranced_high_amt_application'] = prev_app['NFLAG_INSURED_ON_APPROVAL'] * prev_app['HIGH_AMT_APPLICATION']

    prev_app['ANNUITY_PAYMENT_PRODUCT'] = prev_app['AMT_ANNUITY'] * prev_app['CNT_PAYMENT']
    prev_app['HIGH_DOWN_PAYMENT'] = (prev_app['AMT_DOWN_PAYMENT'] > 10000)

    prev_app['CREDIT_DOWNPAYMENT'] = prev_app['AMT_GOODS_PRICE'] - prev_app['AMT_DOWN_PAYMENT']

    prev_app['CREDIT_ANNUITY_RATIO'] = prev_app['AMT_CREDIT'] / prev_app['AMT_ANNUITY'] * (prev_app['AMT_ANNUITY'] != 0)
    prev_app['RATIO_APPLICATION_TO_ANNUITY'] =  prev_app['AMT_APPLICATION'] / prev_app['AMT_ANNUITY'] * (prev_app['AMT_ANNUITY'] != 0)
    prev_app['RATIO_GOODS_TO_ANNUITY'] =  prev_app['AMT_GOODS_PRICE'] / prev_app['AMT_ANNUITY'] * (prev_app['AMT_ANNUITY'] != 0)

    # v1.2
    mask_AMT_ANNUITY = (prev_app['AMT_ANNUITY'] == 0) 
    prev_app.loc[mask_AMT_ANNUITY, 'CREDIT_ANNUITY_RATIO'] = np.nan
    prev_app.loc[mask_AMT_ANNUITY, 'RATIO_APPLICATION_TO_ANNUITY'] = np.nan
    prev_app.loc[mask_AMT_ANNUITY, 'RATIO_GOODS_TO_ANNUITY'] = np.nan

    #Churn_prev: Whether the payment date is late 
    # 1= delayed, 0 = not delayed, NaN = null
    k = prev_app.DAYS_LAST_DUE_1ST_VERSION - prev_app.DAYS_LAST_DUE
    prev_app["CHURN_PREV"] = [1 if i >= 0 else (0 if i < 0  else "NaN") for i in k]

    prev_app['AMT_DIFF_PERCENT'] = ((prev_app['AMT_CREDIT'] - prev_app['AMT_APPLICATION']) / prev_app['AMT_APPLICATION']) * 100
    prev_app['ADJUSTMENT_DIRECTION'] = (prev_app['AMT_CREDIT'] - prev_app['AMT_APPLICATION']) >= 0
    prev_app['ADJUSTMENT_DIRECTION'] = prev_app['ADJUSTMENT_DIRECTION'].map({'True': 1, 'False': 0}) #whether final credit amount is equal clients' application 

    #Interest rate
    prev_app['INTEREST'] = prev_app['CNT_PAYMENT']*prev_app['AMT_ANNUITY'] - prev_app['AMT_CREDIT']
    prev_app['INTEREST_RATE'] = 2*12*prev_app['INTEREST']/(prev_app['AMT_CREDIT']*(prev_app['CNT_PAYMENT']+1))
    prev_app['INTEREST_SHARE'] = prev_app['INTEREST']/prev_app['AMT_CREDIT']


    prev_app["DAYS_DECISION"] = [1 if abs(i/(12*30)) <=1 else 0 for i in prev_app.DAYS_DECISION]

    #Separate weekend from weekdays
    prev_app["WEEKDAY_APPR_PROCESS_START"] = prev_app["WEEKDAY_APPR_PROCESS_START"].replace(['MONDAY','TUESDAY', 'WEDNESDAY','THURSDAY','FRIDAY'], 'WEEK_DAY')
    prev_app["WEEKDAY_APPR_PROCESS_START"] = prev_app["WEEKDAY_APPR_PROCESS_START"].replace(['SATURDAY', 'SUNDAY'], 'WEEKEND')
    prev_app["WEEKDAY_APPR_PROCESS_START"].fillna('WEEK_DAY', inplace=True)

    #Separate working hours from off_hours
    prev_app["HOUR_APPR_PROCESS_START"] = prev_app["HOUR_APPR_PROCESS_START"].replace([8,9,10,11,12,13,14,15,16,17], 'working_hours')
    prev_app["HOUR_APPR_PROCESS_START"] = prev_app["HOUR_APPR_PROCESS_START"].replace([18,19,20,21,22,23,0,1,2,3,4,5,6,7], 'off_hours')

    prev_app['PRODUCT_COMBINATION'].replace(['XNA', 'POS others without interest'], np.nan, inplace=True)
    prev_app['NAME_CONTRACT_TYPE'].replace('XNA', np.nan, inplace=True)
    prev_app['NAME_CLIENT_TYPE'].replace('XNA', np.nan, inplace=True)

    prev_app['NAME_PAYMENT_TYPE'].replace('Cashless from the account of the employer', np.nan, inplace=True)
    prev_app['NAME_GOODS_CATEGORY'].replace(good_replace_2, np.nan, inplace=True)
    prev_app['NAME_CASH_LOAN_PURPOSE'].replace(['Everyday expenses', 'Buying a car', 'Repairs'], np.nan, inplace=True)

    prev_app['CHANNEL_TYPE'].replace('Car dealer', np.nan, inplace=True)
    prev_app['NAME_SELLER_INDUSTRY'].replace(['Tourism', 'Jewelry', 'MLM partners'], np.nan, inplace=True)
    prev_app['CODE_REJECT_REASON'].replace(['XNA', 'CLIENT'], np.nan, inplace=True)
    prev_app['NAME_PORTFOLIO'].replace('Cars', np.nan, inplace=True)

    prev_app, cat_cols = OHE(prev_app, nan_as_category=False)

    cat_aggregations = {}
    cat_approve_aggegations = {}
    for cat in cat_cols:
        if 'NAME_CONTRACT' in cat:
            
            
            if 'NAME_CONTRACT_STATUS' not in cat and 'Revolving loans' not in cat:
                cat_approve_aggegations[cat] = ['sum']
            if 'XNA' in cat:
                continue

            if 'Unused' in cat:
                cat_aggregations[cat] = ['mean']
            else:
                cat_aggregations[cat] = ['mean', 'sum']

        if 'WEEKDAY' in cat or 'HOUR' in cat:
            cat_aggregations[cat] = ['mean', 'sum']
            cat_approve_aggegations[cat] = ['sum']

        if 'CHUN_PREV' in cat:
            cat_aggregations[cat] = ['sum', 'mean']
            cat_approve_aggegations[cat] = ['sum']
        
        if 'PURPOSE' in cat:
            cat_aggregations[cat] = ['sum']
            cat_approve_aggegations[cat] = ['sum']

        if 'CODE_REJECT_REASON' in cat:
            cat_aggregations[cat] = ['sum']
            
        if cat not in cat_aggregations:
            cat_aggregations[cat] = ['mean']


    num_aggregations = {
                        'AMT_ANNUITY':             ['sum','min', 'max', 'mean', 'std', 'skew'],
                        'AMT_APPLICATION':         ['sum','min', 'max', 'mean', 'std', 'skew'],
                        'AMT_CREDIT':              ['sum','min', 'max', 'mean', 'std', 'skew'],
                        'APP_CREDIT_PERC':         ['sum','min', 'max', 'mean', 'var', 'skew'],
                        'AMT_DOWN_PAYMENT':        [ 'sum','min', 'max', 'mean', 'std', 'skew'],
                        'AMT_GOODS_PRICE':         ['sum','min', 'max', 'mean', 'std', 'skew'],
                        'CREDIT_ANNUITY_RATIO':    ['mean', 'std' ],

                        'DOWN_PAYMENT_RATIO':      ['mean', 'std'],
                        'GOODS_PRICE_RATIO':       ['mean', 'std'],
                        
                        # 'HOUR_APPR_PROCESS_START': ['min', 'max', 'mean'],
                        'RATE_DOWN_PAYMENT':       ['min', 'max', 'mean', 'var'],
                        'DAYS_DECISION':           ['max', 'mean', 'var'],
                        'CNT_PAYMENT':             ['mean', 'sum'],

                        'HIGH_DOWN_PAYMENT':      ['sum'],
                        'ANNUITY_PAYMENT_PRODUCT': ['sum', 'mean', 'std', ],
                        'RATIO_APPLICATION_TO_ANNUITY': ['mean', 'std'],
                        'RATIO_GOODS_TO_ANNUITY': ['mean', 'std'],
                        }

    num_refuse_aggregations = {
                        'AMT_ANNUITY':             ['sum','min', 'max', 'mean', 'std', 'skew'],
                        'AMT_APPLICATION':         ['sum','min', 'max', 'mean', 'std', 'skew'],
                        'AMT_CREDIT':              ['sum','min', 'max', 'mean', 'std', 'skew'],
                        'APP_CREDIT_PERC':         [ 'min', 'max', 'mean', 'var'],
                        'AMT_DOWN_PAYMENT':        [ 'sum','min', 'max', 'mean', 'std'],
                        'AMT_GOODS_PRICE':         ['sum','min', 'max', 'mean', 'std', 'skew'],
                        'CREDIT_ANNUITY_RATIO':    ['mean', 'std' ],
                        
                        # 'HOUR_APPR_PROCESS_START': ['min', 'max', 'mean'],
                        'RATE_DOWN_PAYMENT':       ['min', 'max', 'mean'],
                        'DAYS_DECISION':           [ 'max', 'mean'],
                        'CNT_PAYMENT':             ['mean', 'sum'],

                        'ANNUITY_PAYMENT_PRODUCT': ['sum', 'mean', 'std', ],
                        'RATIO_APPLICATION_TO_ANNUITY': ['mean', 'std'],
                            'RATIO_GOODS_TO_ANNUITY': ['mean', 'std'],
                            
                        }

    grouped = prev_app.groupby('SK_ID_CURR')
    prev_agg=grouped.agg({**num_aggregations, **cat_aggregations})
    prev_agg.columns = pd.Index(['PREV_' + e[0] + "_" + e[1].upper() for e in prev_agg.columns.tolist()])
    prev_agg['PREV_NAME_CONTRACT_STATUS_Refused_SUM'] -= prev_agg['PREV_CODE_REJECT_REASON_SYSTEM_SUM']
    prev_agg.drop('PREV_CODE_REJECT_REASON_SYSTEM_SUM', axis=1, inplace=True)


    num_approve_aggregations = {
                        'AMT_ANNUITY':             ['sum', 'min', 'max',  'mean', 'std', 'skew'],
                        'AMT_APPLICATION':         ['sum','min', 'max', 'mean', 'std', 'skew'],
                        'AMT_CREDIT':              ['sum','min', 'max',  'mean', 'std', 'skew'],
                        'APP_CREDIT_PERC':         ['sum','min', 'max', 'mean', 'var', 'skew'],
                        'AMT_DOWN_PAYMENT':        ['sum','min', 'max', 'mean', 'std', 'skew'],
                        'AMT_GOODS_PRICE':         ['sum','min', 'max', 'mean', 'std',],
                        #'HOUR_APPR_PROCESS_START': ['min', 'max', 'mean'],
                        'RATE_DOWN_PAYMENT':       ['min', 'max', 'mean', 'var'],
                        'CREDIT_DOWNPAYMENT':      ['sum','min', 'max', 'mean'],

                        'DOWN_PAYMENT_RATIO':      ['mean', 'std', 'max'],
                        'GOODS_PRICE_RATIO':       ['mean', 'std', 'max'],
                        
                        'NFLAG_INSURED_ON_APPROVAL' : ['sum'],
                        'Insuranced_amt_application' : ['sum', 'mean', 'std'],
                        'Insuranced_amt_credit' : ['sum', 'mean', 'std'],
                        #  'Insuranced_high_amt_application' : ['sum', 'mean'],
                        #  'RATE_INTEREST_PRIMARY':   ['min', 'max', 'mean'],
                        #  'RATE_INTEREST_PRIVILEGED':['mean'],
                        'DAYS_DECISION':           ['max'],
                        'CNT_PAYMENT':             ['mean', 'sum'],
                        'ACTIVE':                  [ 'mean'],
                        'DURATION':                ['sum', 'max', 'min' ,'mean', 'std', 'skew'],
                        'ACTUAL_DURATION':         ['sum', 'max', 'min', 'mean', 'std', 'skew'],
                        'LIFETIME_LOAN':           ['sum', 'mean'],
                        'FINISH_RATE':             ['mean', 'std'],

                        'HIGH_DOWN_PAYMENT':      ['sum'],
                        'ANNUITY_PAYMENT_PRODUCT': ['sum', 'mean', 'std', ],
                        'RATIO_APPLICATION_TO_ANNUITY': ['mean', 'std'],
                        'RATIO_GOODS_TO_ANNUITY': ['mean', 'std'],
                        
                        }

    approved = prev_app[prev_app['NAME_CONTRACT_STATUS_Approved'] == 1]
    approved_agg = approved.groupby('SK_ID_CURR').agg({**num_approve_aggregations, **cat_approve_aggegations})
    approved_agg.columns = pd.Index(['APPROVED_' + e[0] + "_" + e[1].upper() for e in approved_agg.columns.tolist()])
    approved_agg['ratio_amt_application_insuranced'] = approved_agg['APPROVED_Insuranced_amt_application_SUM'] / approved_agg['APPROVED_AMT_APPLICATION_SUM'] * (approved_agg['APPROVED_AMT_APPLICATION_SUM'] != 0)
    approved_agg['ratio_amt_credit_insuranced'] = approved_agg['APPROVED_Insuranced_amt_credit_SUM'] / approved_agg['APPROVED_AMT_CREDIT_SUM'] * (approved_agg['APPROVED_AMT_CREDIT_SUM'] != 0)

    prev_agg = prev_agg.join(approved_agg,         
                            how='left',           
                            on='SK_ID_CURR')

    refused = prev_app[prev_app['NAME_CONTRACT_STATUS_Refused'] == 1]
    refused_agg = refused.groupby('SK_ID_CURR').agg(num_refuse_aggregations)
    refused_agg.columns = pd.Index(['REFUSED_' + e[0] + "_" + e[1].upper() for e in refused_agg.columns.tolist()])

    prev_agg = prev_agg.join(                    
                            refused_agg,         
                            how='left',
                            on='SK_ID_CURR'
                            )

    prev_app = pd.read_csv(os.path.join(ROOT, "dseb63_previous_application.csv"))

    numbers_of_applications = [1, 3, 5]
    features = pd.DataFrame({'SK_ID_CURR': prev_app['SK_ID_CURR'].unique()})
    prev_applications_sorted = prev_app.sort_values(['SK_ID_CURR', 'DAYS_DECISION'])
    group_object = prev_applications_sorted.groupby(by=['SK_ID_CURR'])['SK_ID_PREV'].nunique().reset_index()
    group_object.rename(index=str,
                        columns={'SK_ID_PREV': 'previous_application_number_of_prev_application'},
                        inplace=True)
    features = features.merge(group_object, on=['SK_ID_CURR'], how='left')

    prev_applications_sorted['previous_application_prev_was_approved'] = (
            prev_applications_sorted['NAME_CONTRACT_STATUS'] == 'Approved').astype('int')
    group_object = prev_applications_sorted.groupby(by=['SK_ID_CURR'])[
        'previous_application_prev_was_approved'].last().reset_index()
    features = features.merge(group_object, on=['SK_ID_CURR'], how='left')

    prev_applications_sorted['previous_application_prev_was_refused'] = (
            prev_applications_sorted['NAME_CONTRACT_STATUS'] == 'Refused').astype('int')
    group_object = prev_applications_sorted.groupby(by=['SK_ID_CURR'])[
        'previous_application_prev_was_refused'].last().reset_index()
    features = features.merge(group_object, on=['SK_ID_CURR'], how='left')

    for number in numbers_of_applications:
        prev_applications_tail = prev_applications_sorted.groupby(by=['SK_ID_CURR']).tail(number)

        group_object = prev_applications_tail.groupby(by=['SK_ID_CURR'])['CNT_PAYMENT'].mean().reset_index()
        group_object.rename(index=str, columns={
            'CNT_PAYMENT': 'previous_application_term_of_last_{}_credits_mean'.format(number)},
                            inplace=True)
        features = features.merge(group_object, on=['SK_ID_CURR'], how='left')

        group_object = prev_applications_tail.groupby(by=['SK_ID_CURR'])['DAYS_DECISION'].mean().reset_index()
        group_object.rename(index=str, columns={
            'DAYS_DECISION': 'previous_application_days_decision_about_last_{}_credits_mean'.format(number)},
                            inplace=True)
        features = features.merge(group_object, on=['SK_ID_CURR'], how='left')

        group_object = prev_applications_tail.groupby(by=['SK_ID_CURR'])['DAYS_FIRST_DRAWING'].mean().reset_index()
        group_object.rename(index=str, columns={
            'DAYS_FIRST_DRAWING': 'previous_application_days_first_drawing_last_{}_credits_mean'.format(number)},
                            inplace=True)
        features = features.merge(group_object, on=['SK_ID_CURR'], how='left')
        
    features.drop(columns=['previous_application_prev_was_approved'], inplace=True)
    prev_agg = prev_agg.merge(features, on='SK_ID_CURR', how='left')

    return prev_agg


## Bureau

In [9]:
def process_bureau_balance():
    bureau_balance = pd.read_csv(os.path.join(ROOT, "dseb63_bureau_balance.csv"))
    bureau_balance['MAX_MONTHS_BALANCE'] = bureau_balance.groupby('SK_ID_BUREAU')['MONTHS_BALANCE'].transform('max')
    bureau_balance['TOTAL_MONTHS'] = bureau_balance.groupby('SK_ID_BUREAU')['MONTHS_BALANCE'].transform('count')
    bureau_balance['DIFF'] = bureau_balance['MAX_MONTHS_BALANCE'] - bureau_balance['MONTHS_BALANCE']
    
    bureau_balance_last_12 = bureau_balance[bureau_balance['DIFF'] <= 24]
    bureau_balance_last_12['C Last 24 Month'] = np.where(bureau_balance_last_12['STATUS'] == 'C', 1, 0)
    bureau_balance_last_12['X Last 24 Month'] = np.where(bureau_balance_last_12['STATUS'] == 'X', 1, 0)
    bureau_balance_last_12['0 Last 24 Month'] = np.where(bureau_balance_last_12['STATUS'] == '0', 1, 0)
    bureau_balance_last_12['1 Last 24 Month'] = np.where(bureau_balance_last_12['STATUS'] == '1', 1, 0)
    bureau_balance_last_12['5 Last 24 Month'] = np.where(bureau_balance_last_12['STATUS'] == '5', 1, 0)

    bureau_balance_last_12['2-4 Last 24 Month'] = np.where((bureau_balance_last_12['STATUS'] != 'C') & (bureau_balance_last_12['STATUS'] != 'X') & (bureau_balance_last_12['STATUS'] != '0')&(bureau_balance_last_12['STATUS'] != '1')&(bureau_balance_last_12['STATUS'] != '5'), 1, 0)

    gb_bureau_balance_last_12 = bureau_balance_last_12.groupby('SK_ID_BUREAU').agg({ 'C Last 24 Month': 'sum', 'X Last 24 Month': 'sum', '0 Last 24 Month': 'sum', '1 Last 24 Month':'sum', '5 Last 24 Month':'sum', '2-4 Last 24 Month': 'sum' })


    gb_max_month = bureau_balance.groupby('SK_ID_BUREAU')['MONTHS_BALANCE'].max()
    gb_max_month.rename('MAX_MONTHS_BALANCE', inplace=True)
    gb_total_months = bureau_balance.groupby('SK_ID_BUREAU')['MONTHS_BALANCE'].count()
    gb_total_months.rename('TOTAL_MONTHS', inplace=True)
    gb_C = bureau_balance[bureau_balance['STATUS'] == 'C'].groupby('SK_ID_BUREAU')['STATUS'].count()
    gb_C.rename('Ratio C', inplace=True)

    gb_X = bureau_balance[bureau_balance['STATUS'] == 'X'].groupby('SK_ID_BUREAU')['STATUS'].count()
    gb_X.rename('Ratio X', inplace=True)

    gb_0 = bureau_balance[bureau_balance['STATUS'] == '0'].groupby('SK_ID_BUREAU')['STATUS'].count()
    gb_0.rename('Ratio 0', inplace=True)

    gb_Other = bureau_balance[(bureau_balance['STATUS'] != 'C') & (bureau_balance['STATUS'] != 'X') & (bureau_balance['STATUS'] != '0')].groupby('SK_ID_BUREAU')['STATUS'].count()
    gb_Other.rename('Ratio Other', inplace=True)
    
    gb_bureau_balance = pd.merge(gb_bureau_balance_last_12, gb_max_month, on='SK_ID_BUREAU')
    gb_bureau_balance = pd.merge(gb_bureau_balance, gb_total_months, on='SK_ID_BUREAU')
    gb_bureau_balance = pd.merge(gb_bureau_balance, gb_C, on='SK_ID_BUREAU')
    gb_bureau_balance = pd.merge(gb_bureau_balance, gb_X, on='SK_ID_BUREAU')
    gb_bureau_balance = pd.merge(gb_bureau_balance, gb_0, on='SK_ID_BUREAU')
    gb_bureau_balance = pd.merge(gb_bureau_balance, gb_Other, on='SK_ID_BUREAU')
    gb_bureau_balance = gb_bureau_balance.reset_index()
    
    return gb_bureau_balance

In [10]:
def bureau_active_close_feat(bureau_active, status = 'ACTIVE'):    
    bureau_active["LAST_30_DAYS"] = np.where(bureau_active['DAYS_CREDIT'] >= -30, 1, 0)
    bureau_active["LAST_60_DAYS"] = np.where(bureau_active['DAYS_CREDIT'] >= -60, 1, 0)
    bureau_active["LAST_90_DAYS"] = np.where(bureau_active['DAYS_CREDIT'] >= -90, 1, 0)
    bureau_active["LAST_180_DAYS"] = np.where(bureau_active['DAYS_CREDIT'] >= -180, 1, 0)
    bureau_active["LAST_365_DAYS"] = np.where(bureau_active['DAYS_CREDIT'] >= -365, 1, 0)
    bureau_active["LAST_1000_DAYS"] = np.where(bureau_active['DAYS_CREDIT'] >= -1000, 1, 0)
    
    bureau_active["NEXT_30_DAYS"] = np.where(bureau_active['DAYS_CREDIT'] <= 30, 1, 0)
    bureau_active["NEXT_180_DAYS"] = np.where(bureau_active['DAYS_CREDIT'] <= 60, 1, 0)
    bureau_active["LIFE_TIME_CREDIT"] = np.where(bureau_active['DAYS_CREDIT_ENDDATE'] >= 16000, 1, 0)

    gb_bureau_active_count = bureau_active.groupby('SK_ID_CURR').agg({ 'SK_ID_BUREAU':'count', 
                                                                      'LAST_30_DAYS': 'sum', 
                                                                      'LAST_60_DAYS': 'sum', 
                                                                      'LAST_90_DAYS': 'sum', 
                                                                      'LAST_180_DAYS': 'sum', 
                                                                      'LAST_365_DAYS': 'sum', 
                                                                      'LAST_1000_DAYS':'sum', 
                                                                      'NEXT_30_DAYS': 'sum', 
                                                                      'NEXT_180_DAYS': 'sum', 
                                                                      'LIFE_TIME_CREDIT': 'sum', 
                                                                      'CREDIT_TYPE':'nunique', 
                                                                      'DAYS_CREDIT': 'max',
                                                                      
                                                                      'AMT_CREDIT_SUM':'sum', 
                                                                      'AMT_CREDIT_MEAN':'mean',
                                                                      'AMT_CREDIT_SUM_DEBT':'sum',
                                                                      
                                                                    'AMT_CREDIT_DEBT_MEAN':'mean',
                                                                      'AMT_CREDIT_SUM_LIMIT':'sum', 
                                                                      'AMT_ANNUITY':'sum'}).reset_index()
    gb_bureau_active_count.rename(columns={ 'SK_ID_BUREAU':status, 'LAST_30_DAYS':f'{status}_LAST_30_DAYS', 'LAST_60_DAYS':f'{status}_LAST_60_DAYS', 'LAST_90_DAYS':f'{status}_LAST_90_DAYS', 'LAST_180_DAYS':f'{status}_LAST_180_DAYS', 'LAST_365_DAYS':f'{status}_LAST_365_DAYS', 'LAST_1000_DAYS':f'{status}_LAST_1000_DAYS', 'NEXT_30_DAYS':f'{status}_NEXT_30_DAYS', 'NEXT_180_DAYS':f'{status}_NEXT_180_DAYS', 'LIFE_TIME_CREDIT':f'{status}_LIFE_TIME_CREDIT', 'CREDIT_TYPE':f'{status}_CREDIT_TYPE', 'AMT_CREDIT_SUM': f'{status}_AMT_CREDIT_SUM', 'AMT_CREDIT_SUM_LIMIT':f'{status}_AMT_CREDIT_SUM_LIMIT', 'AMT_ANNUITY':f'{status}_AMT_ANNUITY', 'DAYS_CREDIT':f'{status}_DAYS_CREDIT' }, inplace=True)
    df_30_days = bureau_active[bureau_active['LAST_30_DAYS'] == 1].copy()
    gb_bureau_active_30_days = df_30_days.groupby('SK_ID_CURR').agg({'AMT_CREDIT_SUM': 'sum'}).reset_index()
    gb_bureau_active_30_days.rename(columns={'AMT_CREDIT_SUM':f'{status}_SUM_CREDIT_30_DAYS'}, inplace=True)

    df_180_days = bureau_active[bureau_active['LAST_180_DAYS'] == 1].copy()
    gb_bureau_active_180_days = df_180_days.groupby('SK_ID_CURR').agg({'AMT_CREDIT_SUM': 'sum', 'AMT_CREDIT_SUM_DEBT': 'sum'}).reset_index()
    gb_bureau_active_180_days.rename(columns={'AMT_CREDIT_SUM':f'{status}_SUM_CREDIT_180_DAYS', 'AMT_CREDIT_SUM_DEBT':f'{status}_SUM_DEBT_180_DAYS'}, inplace=True)


    df_life_time_credit = bureau_active[bureau_active['LIFE_TIME_CREDIT'] == 1].copy()
    
    if status == 'ACTIVE':
        gb_bureau_active_life_time_credit = df_life_time_credit.groupby('SK_ID_CURR').agg({'AMT_CREDIT_SUM': 'sum', 'AMT_CREDIT_SUM_DEBT': 'sum'}).reset_index()
        gb_bureau_active_life_time_credit.rename(columns={'AMT_CREDIT_SUM':f'{status}_SUM_CREDIT_LIFE_TIME', 'AMT_CREDIT_SUM_DEBT':f'{status}_SUM_DEBT_LIFE_TIME'}, inplace=True)
    else:
        gb_bureau_active_life_time_credit = df_life_time_credit.groupby('SK_ID_CURR').agg({'AMT_CREDIT_SUM': 'sum',}).reset_index()
        gb_bureau_active_life_time_credit.rename(columns={'AMT_CREDIT_SUM':f'{status}_SUM_CREDIT_LIFE_TIME'}, inplace=True)

    df_not_life_time_credit = bureau_active[(bureau_active['LIFE_TIME_CREDIT'] == 0) & (bureau_active['DURATION'] >= 0)].copy()
    gb_bureau_active_not_life_time_credit = df_not_life_time_credit.groupby('SK_ID_CURR').agg({'DURATION':'mean'}).reset_index()
    gb_bureau_active_not_life_time_credit.rename(columns={'DURATION':f'{status}_MEAN_DURATION_NOT_LIFE_TIME'}, inplace=True)

    if status == 'ACTIVE':
       
        gb_bureau_active_count = pd.merge(gb_bureau_active_count, gb_bureau_active_30_days, on='SK_ID_CURR', how='left')
    gb_bureau_active_count = pd.merge(gb_bureau_active_count, gb_bureau_active_180_days, on='SK_ID_CURR', how='left')
    gb_bureau_active_count = pd.merge(gb_bureau_active_count, gb_bureau_active_life_time_credit, on='SK_ID_CURR', how='left')
    gb_bureau_active_count = pd.merge(gb_bureau_active_count, gb_bureau_active_not_life_time_credit, on='SK_ID_CURR', how='left')
    
    if status == 'CLOSE':
        
        gb_bureau_active_count.drop(columns=[f'{status}_NEXT_30_DAYS', f'{status}_NEXT_180_DAYS', f'{status}_LAST_30_DAYS', f'{status}_LAST_60_DAYS', f'{status}_SUM_DEBT_180_DAYS'], inplace=True)
        
        # Add latency here
        
        # df_not_life_time_credit['LATENCY_30_DAYS'] = df_not_life_time_credit['LATENCY']*df_not_life_time_credit["LAST_30_DAYS"]
        df_not_life_time_credit['LATENCY_180_DAYS'] = df_not_life_time_credit['LATENCY']*df_not_life_time_credit["LAST_180_DAYS"]
        df_not_life_time_credit['LATENCY_365_DAYS'] = df_not_life_time_credit['LATENCY']*df_not_life_time_credit["LAST_365_DAYS"]
        
        gb_bureau_active_latency = df_not_life_time_credit.groupby('SK_ID_CURR').agg({'LATENCY_180_DAYS':'sum', 'LATENCY_365_DAYS':'sum', 'LATENCY':'mean'}).reset_index()
        gb_bureau_active_latency.rename(columns={'LATENCY_180_DAYS':'CLOSE_LATENCY_180_DAYS', 'LATENCY_365_DAYS':'CLOSE_LATENCY_365_DAYS', 'LATENCY':'CLOSE_MEAN_LATENCY'}, inplace=True)
        gb_bureau_active_count = pd.merge(gb_bureau_active_count, gb_bureau_active_latency, on='SK_ID_CURR', how='left')
        
        gb_bureau_active_count['CLOSE_LATENCY_180_DAYS'] = gb_bureau_active_count['CLOSE_LATENCY_180_DAYS']/gb_bureau_active_count['CLOSE_LAST_180_DAYS']
        gb_bureau_active_count['CLOSE_LATENCY_365_DAYS'] = gb_bureau_active_count['CLOSE_LATENCY_365_DAYS']/gb_bureau_active_count['CLOSE_LAST_365_DAYS']
        
        # Add mask ??
    
    return gb_bureau_active_count

In [11]:
def process_bureau():
    bureau = pd.read_csv(os.path.join(ROOT, "dseb63_bureau.csv"))
    
    bureau['DAYS_CREDIT_ENDDATE'][bureau['DAYS_CREDIT_ENDDATE'] < -40000] = np.nan
    bureau['DAYS_CREDIT_UPDATE'][bureau['DAYS_CREDIT_UPDATE'] < -40000] = np.nan
    bureau['DAYS_ENDDATE_FACT'][bureau['DAYS_ENDDATE_FACT'] < -40000] = np.nan
    
    bureau['CREDIT_DURATION'] = bureau['DAYS_CREDIT'] - bureau['DAYS_CREDIT_ENDDATE']
    bureau['ENDDATE_DIF'] = bureau['DAYS_CREDIT_ENDDATE'] - bureau['DAYS_ENDDATE_FACT']
    bureau['DEBT_PERCENTAGE'] = bureau['AMT_CREDIT_SUM'] / (bureau['AMT_CREDIT_SUM_DEBT'] + 1)
    bureau['DEBT_CREDIT_DIFF'] = bureau['AMT_CREDIT_SUM'] - bureau['AMT_CREDIT_SUM_DEBT']
    bureau['CREDIT_TO_ANNUITY_RATIO'] = bureau['AMT_CREDIT_SUM'] / (bureau['AMT_ANNUITY'] + 1)
    bureau['BUREAU_CREDIT_DEBT_RATIO'] = bureau['AMT_CREDIT_SUM_DEBT'] / (bureau['AMT_CREDIT_SUM'] + 1) 

    bureau['UTILIZATION_RATIO'] = bureau['AMT_CREDIT_SUM_DEBT'] / (bureau['AMT_CREDIT_SUM_LIMIT'] +1)
    
    bureau['DEBT_PERCENTAGE'] *= (bureau['AMT_CREDIT_SUM_DEBT'] !=0)
    bureau['CREDIT_TO_ANNUITY_RATIO'] *= (bureau['AMT_ANNUITY'] !=0)
    bureau['BUREAU_CREDIT_DEBT_RATIO'] *= (bureau['AMT_CREDIT_SUM'] !=0)
    bureau['UTILIZATION_RATIO'] *= (bureau['AMT_CREDIT_SUM_LIMIT'] !=0)

    mask_sum_debt = (bureau['AMT_CREDIT_SUM_DEBT'] ==0)
    bureau.loc[mask_sum_debt, 'DEBT_PERCENTAGE'] = np.nan

    mask_annuity = (bureau['AMT_ANNUITY'] ==0)
    bureau.loc[mask_annuity, 'CREDIT_TO_ANNUITY_RATIO'] = np.nan

    mask_credit_sum = (bureau['AMT_CREDIT_SUM'] ==0)
    bureau.loc[mask_credit_sum, 'BUREAU_CREDIT_DEBT_RATIO'] = np.nan

    mask_credit_sum_limit = (bureau['AMT_CREDIT_SUM_LIMIT'] ==0)
    bureau.loc[mask_credit_sum_limit, 'UTILIZATION_RATIO'] = np.nan

    bureau['DEBT_PERCENTAGE'] = bureau['DEBT_PERCENTAGE'].clip(-9,9)
    bureau['CREDIT_TO_ANNUITY_RATIO'] = bureau['CREDIT_TO_ANNUITY_RATIO'].clip(-9,9)
    bureau['BUREAU_CREDIT_DEBT_RATIO'] = bureau['BUREAU_CREDIT_DEBT_RATIO'].clip(-9,9)
    bureau['UTILIZATION_RATIO'] = bureau['UTILIZATION_RATIO'].clip(-9,9)
    bureau['UNPAID_CREDIT'] = np.where(bureau['DAYS_ENDDATE_FACT'].isnull(), 0, 1) 
    
    bureau['AMT_CREDIT_MEAN'] = bureau['AMT_CREDIT_SUM']
    bureau['AMT_CREDIT_DEBT_MEAN'] = bureau['AMT_CREDIT_SUM_DEBT']
    
    gb_bureau_general = bureau.groupby('SK_ID_CURR').agg({'SK_ID_BUREAU': 'count',
                                                      'AMT_CREDIT_SUM': ['sum', 'mean', 'std'], 
                                                      'AMT_CREDIT_SUM_DEBT': ['sum', 'mean', 'std'],  
                                                      'AMT_CREDIT_SUM_OVERDUE': ['sum'], 
                                                      
                                                      'AMT_ANNUITY': ['sum', 'mean', 'std'], 
                                                       
                                                       # Check later (v3)
                                                      'CNT_CREDIT_PROLONG': [ 'mean'], 
                                                      'DAYS_CREDIT_UPDATE': ['mean', 'max'],
                                                      'CREDIT_DURATION': ['mean', 'std'],
                                                      'AMT_CREDIT_MAX_OVERDUE': ['sum', 'mean', 'std'],
                                                      'AMT_CREDIT_SUM_LIMIT':['sum', 'mean', 'std'], 
                                                      
                                                      # 'CREDIT_DURATION': 'mean',
                                                      # 'AMT_CREDIT_SUM_LIMIT':['mean', 'std'],
                                                      
                                                      # 'CREDIT_CURRENCY': 'nunique', 
                                                      'CREDIT_TYPE': 'nunique', 
                                                      'CREDIT_ACTIVE': 'nunique' ,
                                                      


                                                      'DAYS_CREDIT': ['min', 'max', 'mean', 'var'],
                                                      'ENDDATE_DIF': 'mean',
                                                        'DEBT_PERCENTAGE': ['mean','std'],
                                                        'DEBT_CREDIT_DIFF': ['mean','std'],

                                                          #v3
                                                        # 'CREDIT_TO_ANNUITY_RATIO': 'mean',
                                                        # 'BUREAU_CREDIT_DEBT_RATIO': 'mean',
                                                      
                                                      # v3.1
                                                      'CREDIT_TO_ANNUITY_RATIO': ['mean', 'var'],
                                                      'BUREAU_CREDIT_DEBT_RATIO': ['mean','var'],

                                                        'UTILIZATION_RATIO': ['mean', 'std']
                                                      }).reset_index()
    gb_bureau_general.columns = ['GENERAL_'+'_'.join(col).strip() for col in gb_bureau_general.columns.values]


    gb_bureau_general.rename(columns={'GENERAL_SK_ID_CURR_':'SK_ID_CURR'}, inplace=True)

    gb_bureau_general['GENERAL_AVG_PAST_LOAN_PER_TYPE'] = gb_bureau_general['GENERAL_CREDIT_TYPE_nunique'] / gb_bureau_general['GENERAL_SK_ID_BUREAU_count'] * (gb_bureau_general['GENERAL_SK_ID_BUREAU_count'] != 0)


    others_loan = ['Another type of loan', 'Unknown type of loan', 'Loan for working capital replenishment', 'Cash loan (non-earmarked)', 'Real estate loan', 'Loan for the purchase of equipment', 'Mobile operator loan', 'Interbank credit', 'Loan for purchase of shares (margin lending)']
    mapping_others_loan= {x: 'Others' for x in others_loan}
    bureau['CREDIT_TYPE'] = bureau['CREDIT_TYPE'].replace(mapping_others_loan)

    sum_on_car_loan = bureau[bureau['CREDIT_TYPE'] == 'Car loan'].groupby('SK_ID_CURR').agg({'AMT_CREDIT_SUM': 'sum'}).reset_index()
    sum_on_car_loan.rename(columns={'AMT_CREDIT_SUM':'SUM_CREDIT_CAR_LOAN'}, inplace=True)

    sum_on_credit_card = bureau[bureau['CREDIT_TYPE'] == 'Credit card'].groupby('SK_ID_CURR').agg({'AMT_CREDIT_SUM': 'sum', 'AMT_CREDIT_SUM_DEBT': 'sum', 'AMT_CREDIT_SUM_OVERDUE': 'sum'}).reset_index()
    sum_on_credit_card.rename(columns={'AMT_CREDIT_SUM':'SUM_CREDIT_CARD', 'AMT_CREDIT_SUM_DEBT':'SUM_DEBT_CARD', 'AMT_CREDIT_SUM_OVERDUE':'SUM_OVERDUE_CARD' }, inplace=True)

    sum_on_mortgage = bureau[bureau['CREDIT_TYPE'] == 'Mortgage'].groupby('SK_ID_CURR').agg({'AMT_CREDIT_SUM': 'sum', 'AMT_CREDIT_SUM_DEBT': 'sum'}).reset_index()
    sum_on_mortgage.rename(columns={'AMT_CREDIT_SUM':'SUM_MORTGAGE', 'AMT_CREDIT_SUM_DEBT':'SUM_DEBT_MORTGAGE' }, inplace=True)

    sum_on_consumer_credit = bureau[bureau['CREDIT_TYPE'] == 'Consumer credit'].groupby('SK_ID_CURR').agg({'AMT_CREDIT_SUM': 'sum', 'AMT_CREDIT_SUM_DEBT': 'sum', 'AMT_CREDIT_SUM_OVERDUE': 'sum'}).reset_index()
    sum_on_consumer_credit.rename(columns={'AMT_CREDIT_SUM':'SUM_CONSUMER_CREDIT', 'AMT_CREDIT_SUM_DEBT':'SUM_DEBT_CONSUMER_CREDIT', 'AMT_CREDIT_SUM_OVERDUE':'SUM_OVERDUE_CONSUMER_CREDIT' }, inplace=True)

    sum_on_microloan = bureau[bureau['CREDIT_TYPE'] == 'Microloan'].groupby('SK_ID_CURR').agg({'AMT_CREDIT_SUM': 'sum', 'AMT_CREDIT_SUM_DEBT': 'sum' }).reset_index()
    sum_on_microloan.rename(columns={'AMT_CREDIT_SUM':'SUM_MICROLOAN', 'AMT_CREDIT_SUM_DEBT':'SUM_DEBT_MICROLOAN'}, inplace=True)

    gb_bureau_general = pd.merge(gb_bureau_general, sum_on_car_loan, on='SK_ID_CURR', how='left')
    gb_bureau_general = pd.merge(gb_bureau_general, sum_on_credit_card, on='SK_ID_CURR', how='left')
    gb_bureau_general = pd.merge(gb_bureau_general, sum_on_mortgage, on='SK_ID_CURR', how='left')
    gb_bureau_general = pd.merge(gb_bureau_general, sum_on_consumer_credit, on='SK_ID_CURR', how='left')
    gb_bureau_general = pd.merge(gb_bureau_general, sum_on_microloan, on='SK_ID_CURR', how='left')
    
    bureau['DAYS_CREDIT_ENDDATE'] = bureau['DAYS_CREDIT_ENDDATE'].fillna(365243)
    bureau['DURATION'] = bureau['DAYS_CREDIT_ENDDATE'] - bureau['DAYS_CREDIT']
    
    bureau_active = bureau[bureau['CREDIT_ACTIVE'] == 'Active']
    gb_bureau_active_count = bureau_active_close_feat(bureau_active, 'ACTIVE')
    
    bureau_closed = bureau[bureau['CREDIT_ACTIVE'] == 'Closed']
    bureau_closed = bureau_closed[bureau_closed['DAYS_ENDDATE_FACT'] >= -16000]
    bureau_closed['LATENCY'] =  bureau_closed['DAYS_CREDIT_ENDDATE'] - bureau_closed['DAYS_ENDDATE_FACT']
    gb_bureau_close_count = bureau_active_close_feat(bureau_closed, 'CLOSE')
    bureau_closed.dropna(subset=['DAYS_ENDDATE_FACT'], inplace=True)

    bureau_sold = bureau[bureau['CREDIT_ACTIVE'] == 'Sold']
    bureau_sold['SOLD_LAST_1000_DAYS'] = np.where(bureau_sold['DAYS_CREDIT'].values >= -1000, 1, 0)

    bureau_sold['SOLD_FINISHED'] = np.where(bureau_sold['DAYS_ENDDATE_FACT'].isnull(), 0, 1)
    bureau_sold['SOLD_DURATION'] = (bureau_sold['DAYS_ENDDATE_FACT'] - bureau_sold['DAYS_CREDIT']) * bureau_sold['SOLD_FINISHED']

    gb_bureau_sold = bureau_sold.groupby('SK_ID_CURR').agg({ 'SOLD_LAST_1000_DAYS': 'sum', 'SOLD_FINISHED': 'sum', 'SOLD_DURATION':'sum'}).reset_index()
    gb_bureau_sold['SOLD_DURATION'] = gb_bureau_sold['SOLD_DURATION']/(gb_bureau_sold['SOLD_FINISHED'] + 1e-5)
    mask = (gb_bureau_sold['SOLD_FINISHED'] == 0)
    gb_bureau_sold['SOLD_DURATION'] *= (gb_bureau_sold['SOLD_FINISHED'] != 0)
    gb_bureau_sold['SOLD_DURATION'].loc[mask] = np.nan
    
    bureau_bad_debt = bureau[bureau['CREDIT_ACTIVE'] == 'Bad debt']
    bureau_bad_debt['BAD_DEBT_LAST_5_YEAR'] = np.where(bureau_bad_debt['DAYS_CREDIT'].values >= - 356*5, 1, 0)
    bureau_bad_debt['BAD_DEBT_FINISHED'] = np.where(bureau_bad_debt['DAYS_ENDDATE_FACT'].isnull(), 0, 1)
    gb_bureau_bad_debt = bureau_bad_debt.groupby('SK_ID_CURR').agg({ 'BAD_DEBT_LAST_5_YEAR': 'sum', 'BAD_DEBT_FINISHED': 'sum'}).reset_index()
    gb_bureau_general = pd.merge(gb_bureau_general, gb_bureau_active_count, on='SK_ID_CURR', how='left')
    gb_bureau_general = pd.merge(gb_bureau_general, gb_bureau_close_count, on='SK_ID_CURR', how='left')
    gb_bureau_general = pd.merge(gb_bureau_general, gb_bureau_sold, on='SK_ID_CURR', how='left')
    gb_bureau_general = pd.merge(gb_bureau_general, gb_bureau_bad_debt, on='SK_ID_CURR', how='left')

    bureau_balance = process_bureau_balance()
    bureau_balance = pd.merge(bureau_balance, bureau[['SK_ID_CURR', 'SK_ID_BUREAU']], on='SK_ID_BUREAU', how='left')
    bureau_balance.dropna(subset=['SK_ID_CURR'], inplace=True)
    gb_bureau_balance = bureau_balance.groupby('SK_ID_CURR').agg({'0 Last 24 Month':'sum', 'TOTAL_MONTHS':'sum', 'Ratio C':'sum' , 'Ratio X':'sum', 'Ratio 0':'sum', 'Ratio Other': 'sum'}).reset_index()
    gb_bureau_balance['Ratio C'] = gb_bureau_balance['Ratio C']/gb_bureau_balance['TOTAL_MONTHS'] * (gb_bureau_balance['TOTAL_MONTHS'] != 0)
    gb_bureau_balance['Ratio X'] = gb_bureau_balance['Ratio X']/gb_bureau_balance['TOTAL_MONTHS'] * (gb_bureau_balance['TOTAL_MONTHS'] != 0)
    gb_bureau_balance['Ratio 0'] = gb_bureau_balance['Ratio 0']/gb_bureau_balance['TOTAL_MONTHS'] * (gb_bureau_balance['TOTAL_MONTHS'] != 0)
    gb_bureau_balance['Ratio Other'] = gb_bureau_balance['Ratio Other']/gb_bureau_balance['TOTAL_MONTHS'] * (gb_bureau_balance['TOTAL_MONTHS'] != 0)

    gb_bureau_balance.drop(columns=['TOTAL_MONTHS'], inplace=True)
    gb_bureau_general = pd.merge(gb_bureau_general, gb_bureau_balance, on='SK_ID_CURR', how='left')
    
    
    gb_bureau_general['DEBT_CREDIT_RATIO'] = gb_bureau_general['GENERAL_AMT_CREDIT_SUM_DEBT_sum'] / (gb_bureau_general['GENERAL_AMT_CREDIT_SUM_sum'] + 1e-5) * (gb_bureau_general['GENERAL_AMT_CREDIT_SUM_sum'] != 0)
    mask = (gb_bureau_general['GENERAL_AMT_CREDIT_SUM_sum'] == 0)
    gb_bureau_general['DEBT_CREDIT_RATIO'].loc[mask] = np.nan
    
    return gb_bureau_general

## Credit Card Balance

In [12]:
def process_credit_card_balance():
    balance = pd.read_csv(os.path.join(ROOT, "dseb63_credit_card_balance.csv"))
    balance['AMT_DRAWINGS_ATM_CURRENT'][balance['AMT_DRAWINGS_ATM_CURRENT'] < 0] = np.nan
    balance['AMT_DRAWINGS_CURRENT'][balance['AMT_DRAWINGS_CURRENT'] < 0] = np.nan
    balance["ACTIVE"] = balance["NAME_CONTRACT_STATUS"].apply(lambda x: 1 if x == "Active" else 0)
    balance["COMPLETED"] = balance["NAME_CONTRACT_STATUS"].apply(lambda x: 1 if x == "Completed" else 0)
    balance["DEMAND"] = balance["NAME_CONTRACT_STATUS"].apply(lambda x: 1 if x == "Demand" else 0)
    balance["REFUSED"] = balance["NAME_CONTRACT_STATUS"].apply(lambda x: 1 if x == "Refused" else 0)
    balance["SENT_PROPOSAL"] = balance["NAME_CONTRACT_STATUS"].apply(lambda x: 1 if x == "Sent proposal" else 0)
    balance["SIGNED"] = balance["NAME_CONTRACT_STATUS"].apply(lambda x: 1 if x == "Signed" else 0)
    balance['UTILIZATION'] = balance['AMT_BALANCE'] / (balance['AMT_CREDIT_LIMIT_ACTUAL'] + 1e-6)

    # v1.2
    mask = (balance['AMT_CREDIT_LIMIT_ACTUAL'] == 0)
    balance.loc[mask, 'UTILIZATION'] = np.nan

    balance['RATE_OF_PAYBACK'] = balance['AMT_PAYMENT_TOTAL_CURRENT'] / (balance['AMT_INST_MIN_REGULARITY'] + 1e-6) * (balance['AMT_INST_MIN_REGULARITY']!=0)
    balance['PERCENTAGE_OF_MINIMUM_PAYMENTS_MISSED'] = balance['AMT_PAYMENT_CURRENT'] / (balance['AMT_INST_MIN_REGULARITY'] + 1e-6) *  (balance['AMT_INST_MIN_REGULARITY']!=0)

    # v1.2
    mask = (balance['AMT_INST_MIN_REGULARITY'] == 0)
    balance.loc[mask, 'PERCENTAGE_OF_MINIMUM_PAYMENTS_MISSED'] = np.nan
    balance.loc[mask, 'RATE_OF_PAYBACK'] = np.nan # v1.2.1

    balance['Change_in_Credit_Limit'] = balance['AMT_CREDIT_LIMIT_ACTUAL'] - balance['AMT_CREDIT_LIMIT_ACTUAL'].shift(1)
    balance['Change_in_spending'] = balance['AMT_BALANCE'] - balance['AMT_BALANCE'].shift(1)

    balance['MINIMUM_PAYMENTS_ONLY'] = balance['AMT_PAYMENT_CURRENT'] == balance['AMT_INST_MIN_REGULARITY']
    balance['RAPID_ACCOUNT_TURNOVER'] = balance['CNT_DRAWINGS_CURRENT'] > balance['CNT_DRAWINGS_CURRENT'].shift(1)

    balance['SUM_ALL_CNT_DRAWINGS'] = balance['CNT_DRAWINGS_ATM_CURRENT'] + balance['CNT_DRAWINGS_CURRENT'] + balance['CNT_DRAWINGS_OTHER_CURRENT'] + balance['CNT_DRAWINGS_POS_CURRENT']
    balance['SUM_ALL_AMT_DRAWINGS'] = balance['AMT_DRAWINGS_ATM_CURRENT'] + balance['AMT_DRAWINGS_CURRENT'] + balance['AMT_DRAWINGS_OTHER_CURRENT'] + balance['AMT_DRAWINGS_POS_CURRENT']
    balance['RATIO_ALL_AMT_DRAWINGS_TO_ALL_CNT_DRAWINGS'] = balance['SUM_ALL_AMT_DRAWINGS'] / (balance['SUM_ALL_CNT_DRAWINGS'] + 1e-6) * (balance['SUM_ALL_CNT_DRAWINGS'] != 0)

    # v1.2
    mask = (balance['SUM_ALL_CNT_DRAWINGS'] == 0)
    balance.loc[mask, 'RATIO_ALL_AMT_DRAWINGS_TO_ALL_CNT_DRAWINGS'] = np.nan
    balance['RATIO_ALL_AMT_DRAWINGS_TO_ALL_CNT_DRAWINGS'] = balance['RATIO_ALL_AMT_DRAWINGS_TO_ALL_CNT_DRAWINGS'].clip(-9, 9)

    balance['UTILIZATION'] = balance['UTILIZATION'].clip(-9, 9) # v1.2 99->9
    balance['RATE_OF_PAYBACK'] = balance['RATE_OF_PAYBACK'].clip(-9, 9)
    balance['PERCENTAGE_OF_MINIMUM_PAYMENTS_MISSED'] = balance['PERCENTAGE_OF_MINIMUM_PAYMENTS_MISSED'].clip(-9, 9)
    balance["POSITIVE_CREDIT"] = np.where(balance["AMT_BALANCE"] > 0, 1, 0)

    balance["FLAG_6_MONTHS"] = np.where(balance["MONTHS_BALANCE"] >= -6, 1, 0)
    balance["FLAG_12_MONTHS"] = np.where(balance["MONTHS_BALANCE"] >= -12, 1, 0)
    balance["FLAG_36_MONTHS"] = np.where(balance["MONTHS_BALANCE"] >= -36, 1, 0)

    balance["FLAG_UTILIZATION_LESS_25"] = np.where(balance["UTILIZATION"] < 0.25, 1, 0)
    balance["FLAG_UTILIZATION_LESS_50"] = np.where(balance["UTILIZATION"] < 0.50, 1, 0)
    balance["FLAG_UTILIZATION_LESS_75"] = np.where(balance["UTILIZATION"] < 0.75, 1, 0)
    balance["FLAG_UTILIZATION_LESS_100"] = np.where(balance["UTILIZATION"] < 1.00, 1, 0)
    balance["FLAG_UTILIZATION_MORE_100"] = np.where(balance["UTILIZATION"] > 1.00, 1, 0)

    balance['PRINCIPIAL_RATE'] = balance['AMT_RECEIVABLE_PRINCIPAL'] / (balance['AMT_TOTAL_RECEIVABLE'] + 1e-6)
    mask = (balance['AMT_TOTAL_RECEIVABLE'] == 0)
    balance.loc[mask, 'PRINCIPIAL_RATE'] = np.nan
    balance['PRINCIPIAL_RATE'] = balance['PRINCIPIAL_RATE'].clip(-9, 9) #v1.1: 0 -> 99


    gb_balance = balance.groupby('SK_ID_CURR').agg({
        'MONTHS_BALANCE': ['count'],
        'ACTIVE': 'sum',
        # 'COMPLETED': 'sum',
        # 'DEMAND': 'sum',
        # 'REFUSED': 'sum',
        # 'SENT_PROPOSAL': 'sum',
        'AMT_BALANCE': 'sum', # Divide later
        'AMT_CREDIT_LIMIT_ACTUAL': ['mean', 'max', 'min', 'nunique', 'std', 'skew'],
        'AMT_INST_MIN_REGULARITY': ['mean', 'max', 'nunique', 'skew'],
        'AMT_PAYMENT_TOTAL_CURRENT': ['sum', 'mean', 'max', 'min' , 'std', 'skew'],
        
        'UTILIZATION' : 'sum', # Divide later
        'FLAG_UTILIZATION_LESS_25' : ['sum', 'mean'],
        'FLAG_UTILIZATION_LESS_50' : ['sum', 'mean'],
        'FLAG_UTILIZATION_LESS_75' : ['sum', 'mean'],
        'FLAG_UTILIZATION_LESS_100' : ['sum', 'mean'],
        'FLAG_UTILIZATION_MORE_100' : ['sum', 'mean'],
        
        'CNT_DRAWINGS_ATM_CURRENT' : ['mean', 'sum'],
        'CNT_DRAWINGS_CURRENT' : ['mean', 'sum'],
        
        'AMT_DRAWINGS_ATM_CURRENT' : ['sum', 'mean', 'max', 'std'],
        'AMT_DRAWINGS_CURRENT' : ['sum', 'mean', 'max', 'std'],
        'AMT_DRAWINGS_POS_CURRENT' : ['sum', 'mean', 'max', 'std'],
        'AMT_RECEIVABLE_PRINCIPAL' : 'sum', # Divide later
        'PRINCIPIAL_RATE': 'sum', # Divide later
        # 'SK_DPD': 'sum',
        # 'SK_DPD_DEF': 'sum',
        'POSITIVE_CREDIT': 'sum',

        'MINIMUM_PAYMENTS_ONLY' : 'sum',
        'SUM_ALL_CNT_DRAWINGS' : ['sum', 'mean'],
        'SUM_ALL_AMT_DRAWINGS' : ['sum', 'mean'],
        'RAPID_ACCOUNT_TURNOVER' : ['sum', 'mean'],
        'RATIO_ALL_AMT_DRAWINGS_TO_ALL_CNT_DRAWINGS' : ['mean', 'std'],
        
        'RATE_OF_PAYBACK': 'sum', # Divide later
        'PERCENTAGE_OF_MINIMUM_PAYMENTS_MISSED': 'sum', # Divide later
        'Change_in_Credit_Limit': 'sum',
        'Change_in_spending': ['sum', 'mean', 'std'],
        
    }).reset_index()

    gb_balance.columns = ['_'.join(col).strip() for col in gb_balance.columns.values]
    gb_balance.rename(columns={'SK_ID_CURR_': 'SK_ID_CURR'}, inplace=True)

    gb_balance['AMT_BALANCE_sum'] = gb_balance['AMT_BALANCE_sum'] / (gb_balance['POSITIVE_CREDIT_sum']+1)
    gb_balance['UTILIZATION_sum'] = gb_balance['UTILIZATION_sum'] / (gb_balance['POSITIVE_CREDIT_sum']+1)
    gb_balance['AMT_RECEIVABLE_PRINCIPAL_sum'] = gb_balance['AMT_RECEIVABLE_PRINCIPAL_sum'] / (gb_balance['POSITIVE_CREDIT_sum']+1)
    gb_balance['PRINCIPIAL_RATE_sum'] = gb_balance['PRINCIPIAL_RATE_sum'] / (gb_balance['POSITIVE_CREDIT_sum']+1)
    gb_balance['RATE_OF_PAYBACK_sum'] = gb_balance['RATE_OF_PAYBACK_sum'] / (gb_balance['POSITIVE_CREDIT_sum']+1)
    gb_balance['PERCENTAGE_OF_MINIMUM_PAYMENTS_MISSED_sum'] = gb_balance['PERCENTAGE_OF_MINIMUM_PAYMENTS_MISSED_sum'] / (gb_balance['POSITIVE_CREDIT_sum']+1)
    gb_balance['POSITIVE_PERCENT'] = gb_balance['POSITIVE_CREDIT_sum'] / gb_balance['MONTHS_BALANCE_count']

    balance_6_months = balance[balance['FLAG_6_MONTHS'] ==1]
    balance_12_months = balance[balance['FLAG_12_MONTHS'] ==1]
    balance_36_months = balance[balance['FLAG_36_MONTHS'] ==1]

    gb_balance_6_months = balance_6_months.groupby('SK_ID_CURR').agg({

        'RATE_OF_PAYBACK' : ['mean', 'std'],
        'AMT_BALANCE': 'sum', # Divide later
        'AMT_CREDIT_LIMIT_ACTUAL': ['mean', 'std'],
            'AMT_INST_MIN_REGULARITY': ['mean', 'std'],
        'AMT_PAYMENT_TOTAL_CURRENT': 'sum',
        
        'UTILIZATION' : 'sum', # Divide later
        'FLAG_UTILIZATION_LESS_50' : ['sum' ,'mean'],
        'FLAG_UTILIZATION_LESS_100' : 'sum',
        
        'CNT_DRAWINGS_ATM_CURRENT' :  'sum',
        'CNT_DRAWINGS_CURRENT' : 'sum',   
        'AMT_DRAWINGS_CURRENT' : ['sum', 'max'],
        'AMT_RECEIVABLE_PRINCIPAL' : ['sum', 'mean','std'], # Divide later
        'PRINCIPIAL_RATE' : [ 'mean', 'std'], # Divide later
        'POSITIVE_CREDIT': ['sum']
    }).reset_index()

    gb_balance_6_months.columns = ['6_MONTH_'+'_'.join(col).strip() for col in gb_balance_6_months.columns.values]
    gb_balance_6_months['6_MONTH_AMT_BALANCE_sum'] = gb_balance_6_months['6_MONTH_AMT_BALANCE_sum'] / (gb_balance_6_months['6_MONTH_POSITIVE_CREDIT_sum']+1e-6)
    gb_balance_6_months['6_MONTH_UTILIZATION_sum'] = gb_balance_6_months['6_MONTH_UTILIZATION_sum'] / (gb_balance_6_months['6_MONTH_POSITIVE_CREDIT_sum']+1e-6)
    gb_balance_6_months['6_MONTH_POSITIVE_CREDIT_mean'] = gb_balance_6_months['6_MONTH_POSITIVE_CREDIT_sum']/6

    gb_balance_6_months.drop(columns=['6_MONTH_POSITIVE_CREDIT_sum'], inplace=True)
    gb_balance_6_months.rename(columns={'6_MONTH_SK_ID_CURR_':'SK_ID_CURR'}, inplace=True)

    gb_balance_12_months = balance_12_months.groupby('SK_ID_CURR').agg({
        
            'RATE_OF_PAYBACK' : ['mean', 'std'],
        'AMT_BALANCE': 'sum', # Divide later
        'AMT_CREDIT_LIMIT_ACTUAL': ['mean', 'max', 'min', 'std'],
        'AMT_INST_MIN_REGULARITY': ['mean', 'max',  'std'],
        'AMT_PAYMENT_TOTAL_CURRENT': ['sum', 'mean', 'std'],
        
        'UTILIZATION' : 'sum', # Divide later
        'FLAG_UTILIZATION_LESS_50' : 'sum',
        'FLAG_UTILIZATION_LESS_100' : 'sum',
        
        'CNT_DRAWINGS_ATM_CURRENT' :  'sum',
        'CNT_DRAWINGS_CURRENT' : 'sum',   
        'AMT_DRAWINGS_CURRENT' : ['sum', 'max'],

    'AMT_RECEIVABLE_PRINCIPAL' : ['sum', 'mean','std'], # Divide later
    'PRINCIPIAL_RATE' : [ 'mean', 'std', 'max'],
        'POSITIVE_CREDIT': 'sum',
        }).reset_index()

    gb_balance_12_months.columns = ['12_MONTH_'+'_'.join(col).strip() for col in gb_balance_12_months.columns.values]
    gb_balance_12_months['12_MONTH_AMT_BALANCE_sum'] = gb_balance_12_months['12_MONTH_AMT_BALANCE_sum'] / (gb_balance_12_months['12_MONTH_POSITIVE_CREDIT_sum']+1e-6)
    gb_balance_12_months['12_MONTH_UTILIZATION_sum'] = gb_balance_12_months['12_MONTH_UTILIZATION_sum'] / (gb_balance_12_months['12_MONTH_POSITIVE_CREDIT_sum']+1e-6)
    gb_balance_12_months.rename(columns={'12_MONTH_SK_ID_CURR_':'SK_ID_CURR'}, inplace=True)
    gb_balance_12_months['12_MONTH_POSITIVE_CREDIT_mean'] = gb_balance_12_months['12_MONTH_POSITIVE_CREDIT_sum']/12

    gb_balance_36_months = balance_36_months.groupby('SK_ID_CURR').agg({
            
                'AMT_BALANCE': 'sum', # Divide later
            'AMT_CREDIT_LIMIT_ACTUAL': ['mean', 'std'],
            'AMT_INST_MIN_REGULARITY': ['mean', 'std'],
            'AMT_PAYMENT_TOTAL_CURRENT': ['sum', 'mean',  'std', 'skew'],
            
            'UTILIZATION' : 'sum', # Divide later
            'FLAG_UTILIZATION_LESS_50' : 'sum',
            'FLAG_UTILIZATION_LESS_100' : 'sum',
            
            'CNT_DRAWINGS_ATM_CURRENT' :  'sum',
            'CNT_DRAWINGS_CURRENT' : 'sum',   
            'AMT_DRAWINGS_CURRENT' : ['sum', 'mean', 'max'],
        
            'POSITIVE_CREDIT': 'sum',
            'AMT_RECEIVABLE_PRINCIPAL' : ['sum', 'mean','std'], # Divide later
        }).reset_index()

    gb_balance_36_months.columns = ['36_MONTH_'+'_'.join(col).strip() for col in gb_balance_36_months.columns.values]
    gb_balance_36_months['36_MONTH_AMT_BALANCE_sum'] = gb_balance_36_months['36_MONTH_AMT_BALANCE_sum'] / (gb_balance_36_months['36_MONTH_POSITIVE_CREDIT_sum']+1e-6)
    gb_balance_36_months['36_MONTH_UTILIZATION_sum'] = gb_balance_36_months['36_MONTH_UTILIZATION_sum'] / (gb_balance_36_months['36_MONTH_POSITIVE_CREDIT_sum']+1e-6)
    gb_balance_36_months.rename(columns={'36_MONTH_SK_ID_CURR_':'SK_ID_CURR'}, inplace=True)
    gb_balance_36_months['36_MONTH_POSITIVE_CREDIT_mean'] = gb_balance_36_months['36_MONTH_POSITIVE_CREDIT_sum']/36

    gb_balance = pd.merge(gb_balance, gb_balance_6_months, on='SK_ID_CURR', how='left')
    gb_balance = pd.merge(gb_balance, gb_balance_12_months, on='SK_ID_CURR', how='left')
    gb_balance = pd.merge(gb_balance, gb_balance_36_months, on='SK_ID_CURR', how='left')

    return gb_balance

## POS CASH Balance

In [13]:
def process_pos_cash_balance():
    pos_cash = pd.read_csv(os.path.join(ROOT, "dseb63_POS_CASH_balance.csv"))
    pos_cash = pos_cash[pos_cash['NAME_CONTRACT_STATUS'] != 'XNA']
    pos_cash = pos_cash[pos_cash['NAME_CONTRACT_STATUS'] != 'Canceled']
    
    pos_cash['COMPLETED'] = pos_cash['NAME_CONTRACT_STATUS'] == 'Completed'
    pos_cash['COUNT_DEMAND'] = pos_cash['NAME_CONTRACT_STATUS'] == 'Demand'
    pos_cash['COUNT_SIGNED'] = pos_cash['NAME_CONTRACT_STATUS'] == 'Signed'
    pos_cash['COUNT_APPROVED'] = pos_cash['NAME_CONTRACT_STATUS'] == 'Approved'
    pos_cash['COUNT_RETURNED'] = pos_cash['NAME_CONTRACT_STATUS'] == 'Returned to the store'

    pos_cash['LAST_3_MONTHS'] = pos_cash['MONTHS_BALANCE'] >= -3
    pos_cash['LAST_6_MONTHS'] = pos_cash['MONTHS_BALANCE'] >= -6
    pos_cash['LAST_12_MONTHS'] = pos_cash['MONTHS_BALANCE'] >= -12
    pos_cash['LAST_36_MONTHS'] = pos_cash['MONTHS_BALANCE'] >= -36

    pos_cash['NUM_INSTALMENT'] = 1

    pos_cash['COUNT_SK_DPD'] = pos_cash['SK_DPD'] > 0
    pos_cash['COUNT_SK_DPD_DEF'] = pos_cash['SK_DPD_DEF'] > 0

    pos_cash['MEAN_SK_DPD'] = pos_cash['SK_DPD']
    pos_cash['MEAN_SK_DPD_DEF'] = pos_cash['SK_DPD_DEF']

    gb_pos_cash = pos_cash.groupby('SK_ID_PREV').agg({
        'SK_ID_CURR': 'first',
        'NUM_INSTALMENT': 'sum',
        'COMPLETED': 'sum',
        'COUNT_DEMAND': 'sum',
        'COUNT_SIGNED': 'sum',
        'COUNT_APPROVED': 'sum',
        'COUNT_RETURNED': 'sum',
        'CNT_INSTALMENT': 'max',
        'MONTHS_BALANCE': 'max',
        
        'COUNT_SK_DPD':'sum',
        'COUNT_SK_DPD_DEF':'sum',
        
        'SK_DPD':'sum',
        'SK_DPD_DEF':'sum',
        
        'MEAN_SK_DPD':'mean',
        'MEAN_SK_DPD_DEF':'mean',
        
        'CNT_INSTALMENT_FUTURE':'min',
        'LAST_3_MONTHS':'sum',
        'LAST_6_MONTHS':'sum',
        'LAST_12_MONTHS':'sum',
        'LAST_36_MONTHS':'sum',
    }).reset_index()

    gb_pos_cash['LONG_TERM'] = gb_pos_cash['CNT_INSTALMENT'] > 24
    gb_pos_cash['SHORT_TERM'] = gb_pos_cash['CNT_INSTALMENT'] <= 24
        
    gb_pos_cash_by_curr = gb_pos_cash.groupby('SK_ID_CURR').agg({
        'NUM_INSTALMENT': 'sum',
        'COMPLETED': ['sum', 'mean'],
        # 'COUNT_DEMAND': 'sum',
        'COUNT_SIGNED': 'sum',
        # 'COUNT_APPROVED': ['sum', 'mean'],
        # 'COUNT_RETURNED': 'sum',
        'CNT_INSTALMENT': ['sum', 'mean', 'var'],
        'MONTHS_BALANCE': 'max',
        
        'SK_DPD':['sum', 'mean', 'max', 'var'],
        'SK_DPD_DEF':['sum', 'mean', 'max', 'var'],
        
        'COUNT_SK_DPD':['sum', 'max', 'mean', 'var'],
        'COUNT_SK_DPD_DEF':['sum', 'max', 'mean', 'var'],
        
        'MEAN_SK_DPD':['sum', 'mean', 'max', 'var'],
        'MEAN_SK_DPD_DEF':['sum', 'mean', 'max', 'var'],
        
        'CNT_INSTALMENT_FUTURE':'sum',
        
        'LAST_3_MONTHS':'sum',
        'LAST_6_MONTHS':'sum',
        'LAST_12_MONTHS':'sum',
        'LAST_36_MONTHS':'sum',
    }).reset_index()

    gb_pos_cash_by_curr.columns = ['GENERAL_'+'_'.join(col).strip() for col in gb_pos_cash_by_curr.columns.values]
    gb_pos_cash_by_curr.rename(columns={'GENERAL_SK_ID_CURR_':'SK_ID_CURR'}, inplace=True)

    gb_pos_cash_complete = gb_pos_cash[gb_pos_cash['COMPLETED'] > 0]
    gb_pos_cash_not_complete = gb_pos_cash[gb_pos_cash['COMPLETED'] == 0]
    gb_pos_cash_complete['RATE_COMPLETED'] = gb_pos_cash_complete['CNT_INSTALMENT'] / gb_pos_cash_complete['NUM_INSTALMENT']

    gb_pos_cash_complete_long_term = gb_pos_cash_complete[gb_pos_cash_complete['LONG_TERM'] == 1]
    gb_pos_cash_complete_short_term = gb_pos_cash_complete[gb_pos_cash_complete['LONG_TERM'] == 0]

    gb_pos_cash_complete_long_term_by_curr = gb_pos_cash_complete_long_term.groupby('SK_ID_CURR').agg({
        'CNT_INSTALMENT': 'mean',
        'RATE_COMPLETED': 'mean',
        'SK_ID_PREV': 'count',
        'NUM_INSTALMENT': 'sum',
        'LAST_12_MONTHS': 'sum',
        'LAST_36_MONTHS': 'sum',
        # 'SK_DPD':'sum',
        # 'SK_DPD_DEF':'sum',
    }).reset_index()

    gb_pos_cash_complete_long_term_by_curr.rename(columns={col: 'LONG_TERM_' + col for col in gb_pos_cash_complete_long_term_by_curr.columns if col != 'SK_ID_CURR'}, inplace=True)

    gb_pos_cash_complete_short_term_by_curr = gb_pos_cash_complete_short_term.groupby('SK_ID_CURR').agg({
        'CNT_INSTALMENT': 'mean',
        'RATE_COMPLETED': 'mean',
        'SK_ID_PREV': 'count',
        'NUM_INSTALMENT': 'sum',
        'LAST_12_MONTHS': 'sum',
        'LAST_36_MONTHS': 'sum',
        'SK_DPD':'sum',
        'SK_DPD_DEF':'sum',
    }).reset_index()

    gb_pos_cash_complete_short_term_by_curr.rename(columns={col: 'SHORT_TERM_' + col for col in gb_pos_cash_complete_short_term_by_curr.columns if col != 'SK_ID_CURR'}, inplace=True)

    gb_pos_cash_not_complete['SHORT_CNT_INSTALMENT'] = gb_pos_cash_complete_short_term['CNT_INSTALMENT'] * gb_pos_cash_complete_short_term['SHORT_TERM']
    gb_pos_cash_not_complete['LONG_CNT_INSTALMENT'] = gb_pos_cash_complete_long_term['CNT_INSTALMENT'] * gb_pos_cash_complete_long_term['LONG_TERM']

    gb_pos_cash_not_complete_by_curr = gb_pos_cash_not_complete.groupby('SK_ID_CURR').agg({
        # 'SHORT_CNT_INSTALMENT': 'sum',
        # 'LONG_CNT_INSTALMENT' : 'sum',
        'LAST_12_MONTHS': 'sum',
        'LAST_36_MONTHS': 'sum',
        'CNT_INSTALMENT_FUTURE': 'sum',
        'SK_DPD':'sum',
        'SK_DPD_DEF':'sum',
    }).reset_index()

    gb_pos_cash_not_complete_by_curr.rename(columns={col: 'NOT_COMPLETED_' + col for col in gb_pos_cash_not_complete_by_curr.columns if col != 'SK_ID_CURR'}, inplace=True)
    gb_pos_cash_by_curr = gb_pos_cash_by_curr.merge(gb_pos_cash_complete_long_term_by_curr, on='SK_ID_CURR', how='left')
    gb_pos_cash_by_curr = gb_pos_cash_by_curr.merge(gb_pos_cash_complete_short_term_by_curr, on='SK_ID_CURR', how='left')
    gb_pos_cash_by_curr = gb_pos_cash_by_curr.merge(gb_pos_cash_not_complete_by_curr, on='SK_ID_CURR', how='left')

    return gb_pos_cash_by_curr


## Installment Payment

In [14]:
def process_installment_payment():
    installment = pd.read_csv(os.path.join(ROOT, "dseb63_installments_payments.csv"))
    installment['INSTALLMENT_PAYMENT_DIFF'] = installment['AMT_INSTALMENT'] - installment['AMT_PAYMENT']
    installment['DIFF'] = installment['DAYS_INSTALMENT'] - installment['DAYS_ENTRY_PAYMENT']
    # installment['DIFF'].fillna(0, inplace=True)
    installment['LATE'] = installment['DIFF'] > 0
    installment['EARLY'] = installment['DIFF'] <= 0

    installment['LATE'] *= installment['DIFF']
    installment['EARLY'] *= -installment['DIFF']

    installment["LAST_30_DAYS"] = installment["DAYS_ENTRY_PAYMENT"] >= -30
    installment["LAST_60_DAYS"] = installment["DAYS_ENTRY_PAYMENT"] >= -60
    installment["LAST_90_DAYS"] = installment["DAYS_ENTRY_PAYMENT"] >= -90
    installment["LAST_180_DAYS"] = installment["DAYS_ENTRY_PAYMENT"] >= -180
    installment["LAST_365_DAYS"] = installment["DAYS_ENTRY_PAYMENT"] >= -365
    installment["LAST_730_DAYS"] = installment["DAYS_ENTRY_PAYMENT"] >= -365*3 # 3 ye

    installment["PAYMENT_LAST_30_DAYS"] = installment["LAST_30_DAYS"] * installment["AMT_PAYMENT"]
    installment["PAYMENT_LAST_60_DAYS"] = installment["LAST_60_DAYS"] * installment["AMT_PAYMENT"]
    installment["PAYMENT_LAST_90_DAYS"] = installment["LAST_90_DAYS"] * installment["AMT_PAYMENT"]
    installment["PAYMENT_LAST_180_DAYS"] = installment["LAST_180_DAYS"] * installment["AMT_PAYMENT"]
    installment["PAYMENT_LAST_365_DAYS"] = installment["LAST_365_DAYS"] * installment["AMT_PAYMENT"]
    installment["PAYMENT_LAST_730_DAYS"] = installment["LAST_730_DAYS"] * installment["AMT_PAYMENT"]

    installment["LATENESS_LAST_90_DAYS"] = installment["LAST_90_DAYS"] * installment["LATE"]
    installment["LATENESS_LAST_180_DAYS"] = installment["LAST_180_DAYS"] * installment["LATE"]
    installment["LATENESS_LAST_365_DAYS"] = installment["LAST_365_DAYS"] * installment["LATE"]
    installment["LATENESS_LAST_730_DAYS"] = installment["LAST_730_DAYS"] * installment["LATE"]

    installment["EARLYNESS_LAST_90_DAYS"] = installment["LAST_90_DAYS"] * installment["EARLY"]
    installment["EARLYNESS_LAST_180_DAYS"] = installment["LAST_180_DAYS"] * installment["EARLY"]
    installment["EARLYNESS_LAST_365_DAYS"] = installment["LAST_365_DAYS"] * installment["EARLY"]
    installment["EARLYNESS_LAST_730_DAYS"] = installment["LAST_730_DAYS"] * installment["EARLY"]

    installment["PAYMENT_LATENESS_LAST_180_DAYS"] = installment["LAST_180_DAYS"] * installment["LATENESS_LAST_180_DAYS"]
    installment["PAYMENT_LATENESS_LAST_365_DAYS"] = installment["LAST_365_DAYS"] * installment["LATENESS_LAST_365_DAYS"]
    installment["PAYMENT_LATENESS_LAST_730_DAYS"] = installment["LAST_730_DAYS"] * installment["LATENESS_LAST_730_DAYS"]

    installment["PAYMENT_EARLYNESS_LAST_180_DAYS"] = installment["LAST_180_DAYS"] * installment["EARLYNESS_LAST_180_DAYS"]
    installment["PAYMENT_EARLYNESS_LAST_365_DAYS"] = installment["LAST_365_DAYS"] * installment["EARLYNESS_LAST_365_DAYS"]
    installment["PAYMENT_EARLYNESS_LAST_730_DAYS"] = installment["LAST_730_DAYS"] * installment["EARLYNESS_LAST_730_DAYS"]

    installment['INSTALLMENT_PAYMENT_DIFF_LAST_180_DAYS'] = installment['INSTALLMENT_PAYMENT_DIFF'] * installment['LAST_180_DAYS']
    installment['INSTALLMENT_PAYMENT_DIFF_LAST_365_DAYS'] = installment['INSTALLMENT_PAYMENT_DIFF'] * installment['LAST_365_DAYS']
    installment['INSTALLMENT_PAYMENT_DIFF_LAST_730_DAYS'] = installment['INSTALLMENT_PAYMENT_DIFF'] * installment['LAST_730_DAYS']

    installment['COUNT'] = 1

    gb_installment = installment.groupby('SK_ID_PREV').agg({'SK_ID_CURR':'first', 
                                                            'NUM_INSTALMENT_NUMBER':'max', 
                                                            'COUNT':'sum', 
                                                            'DIFF':['sum', 'mean', 'std'],
                                                            'LATE': ['sum', 'mean', 'std'], 
                                                            'EARLY': ['sum', 'mean', 'std'], 
                                                            'DAYS_ENTRY_PAYMENT':'max', 
                                                            'INSTALLMENT_PAYMENT_DIFF' : ['sum', 'mean', 'std'],
                                                            'AMT_PAYMENT':'sum',
                                                            
                                                            'LAST_30_DAYS':'sum', 
                                                            'LAST_60_DAYS':'sum', 
                                                            'LAST_90_DAYS':'sum', 
                                                            'LAST_180_DAYS':'sum', 
                                                            'LAST_365_DAYS':'sum', 
                                                            'LAST_730_DAYS':'sum', 
                                                            
                                                            'PAYMENT_LAST_30_DAYS':['sum', 'mean', 'std'],  # 'max',
                                                            'PAYMENT_LAST_60_DAYS':['sum', 'mean', 'std'], 
                                                            'PAYMENT_LAST_90_DAYS':['sum', 'mean', 'std', 'max','min'], 
                                                            'PAYMENT_LAST_180_DAYS':['sum', 'mean', 'std'], 
                                                            'PAYMENT_LAST_365_DAYS':['sum', 'mean', 'std', 'max','min'], 
                                                            'PAYMENT_LAST_730_DAYS':['sum', 'mean', 'std'],
                                                            
                                                            'LATENESS_LAST_90_DAYS':['sum', 'mean'],
                                                            'LATENESS_LAST_180_DAYS':['sum', 'mean', 'std'],
                                                            'LATENESS_LAST_365_DAYS':['sum', 'mean'],
                                                            'LATENESS_LAST_730_DAYS':['sum', 'mean', 'std'],
                                                            
                                                            'EARLYNESS_LAST_90_DAYS':['sum', 'mean'],
                                                            'EARLYNESS_LAST_180_DAYS':['sum', 'mean', 'std'],
                                                            'EARLYNESS_LAST_365_DAYS':['sum', 'mean'],
                                                            'EARLYNESS_LAST_730_DAYS':['sum', 'mean', 'std'],
                                                            
                                                            'PAYMENT_LATENESS_LAST_180_DAYS':['sum', 'mean'],
                                                            'PAYMENT_LATENESS_LAST_365_DAYS':['sum', 'mean', 'std', 'max'],
                                                            'PAYMENT_LATENESS_LAST_730_DAYS':['sum', 'mean', 'std'],
                                                            
                                                            'PAYMENT_EARLYNESS_LAST_180_DAYS':['sum', 'mean'],
                                                            'PAYMENT_EARLYNESS_LAST_365_DAYS':['sum', 'mean', 'std', 'max'],
                                                            'PAYMENT_EARLYNESS_LAST_730_DAYS':['sum', 'mean', 'std'],
                                                            
                                                            'INSTALLMENT_PAYMENT_DIFF_LAST_180_DAYS':['sum', 'mean', 'std'],
                                                            'INSTALLMENT_PAYMENT_DIFF_LAST_365_DAYS':['sum', 'mean', 'std', 'max'],
                                                            'INSTALLMENT_PAYMENT_DIFF_LAST_730_DAYS':['sum', 'mean', 'std', 'max'],
                                                            
                                                            }).reset_index()

    gb_installment.columns = ['_'.join(col).strip() for col in gb_installment.columns.values]
    # gb_installment.rename(columns={'COUNT':'NUM_INSTALMENT', 'LATE':'LATE_PAYMENT', 'EARLY':'EARLY_PAYMENT', 'DAYS_ENTRY_PAYMENT':'LASTEST_PAYMENT_DATE', 'AMT_PAYMENT':'TOTAL_PAYMENT'}, inplace=True)
    gb_installment['MEAN_INSTALLMENT'] = gb_installment['AMT_PAYMENT_sum'] / gb_installment['NUM_INSTALMENT_NUMBER_max']

    gb_installment.rename({ 'SK_ID_PREV_': 'SK_ID_PREV', 'SK_ID_CURR_first': 'SK_ID_CURR'}, axis=1, inplace=True)
    # gb_mean_installment = gb_installment.groupby('SK_ID_CURR').agg({'MEAN_INSTALLMENT':'mean'}).reset_index()
    gb_installment_by_curr = installment.groupby('SK_ID_CURR').agg({
                                                            'NUM_INSTALMENT_NUMBER':'max', 
                                                            'COUNT':'sum', 
                                                            'DIFF':['sum', 'mean', 'std'],
                                                            'LATE': ['sum', 'mean', 'std'], 
                                                            'EARLY': ['sum', 'mean', 'std'], 
                                                            'DAYS_ENTRY_PAYMENT':'max', 
                                                            'INSTALLMENT_PAYMENT_DIFF' : ['sum', 'mean', 'std'],
                                                            'AMT_PAYMENT':'sum',
                                                            
                                                            'LAST_60_DAYS':'sum', 
                                                            'LAST_90_DAYS':'sum', 
                                                            'LAST_180_DAYS':'sum', 
                                                            'LAST_365_DAYS':'sum', 
                                                            'LAST_730_DAYS':'sum', 
                                                            
                                                            'PAYMENT_LAST_30_DAYS':['sum', 'mean', 'std'], 
                                                            'PAYMENT_LAST_60_DAYS':['sum', 'mean', 'std'], 
                                                            'PAYMENT_LAST_90_DAYS':['sum', 'mean', 'std', 'max'], 
                                                            'PAYMENT_LAST_180_DAYS':['sum', 'mean', 'std'], 
                                                            'PAYMENT_LAST_365_DAYS':['sum', 'mean', 'std', 'max','min'], # drop min ?
                                                            'PAYMENT_LAST_730_DAYS':['sum', 'mean', 'std'],
                                                            
                                                            'LATENESS_LAST_90_DAYS':['sum', 'mean'],
                                                            'LATENESS_LAST_180_DAYS':['sum', 'mean', 'std'],
                                                            'LATENESS_LAST_365_DAYS':['sum', 'mean'],
                                                            'LATENESS_LAST_730_DAYS':['sum', 'mean', 'std', 'max'],
                                                            
                                                            'EARLYNESS_LAST_90_DAYS':['sum', 'mean'],
                                                            'EARLYNESS_LAST_180_DAYS':['sum', 'mean', 'std',],
                                                            'EARLYNESS_LAST_365_DAYS':['sum', 'mean'],
                                                            'EARLYNESS_LAST_730_DAYS':['sum', 'mean', 'std', 'max'],
                                                            
                                                            'PAYMENT_LATENESS_LAST_180_DAYS':['sum', 'mean'],
                                                            'PAYMENT_LATENESS_LAST_365_DAYS':['sum', 'mean', 'std'],
                                                            'PAYMENT_LATENESS_LAST_730_DAYS':['sum', 'mean', 'std', 'max'],
                                                            
                                                            'PAYMENT_EARLYNESS_LAST_180_DAYS':['sum', 'mean'],
                                                            'PAYMENT_EARLYNESS_LAST_365_DAYS':['sum', 'mean', 'std'],
                                                            'PAYMENT_EARLYNESS_LAST_730_DAYS':['sum', 'mean', 'std', 'max'],
                                                            
                                                            'INSTALLMENT_PAYMENT_DIFF_LAST_180_DAYS':['sum', 'mean', 'std'],
                                                            'INSTALLMENT_PAYMENT_DIFF_LAST_365_DAYS':['sum', 'mean', 'std', 'max'],
                                                            'INSTALLMENT_PAYMENT_DIFF_LAST_730_DAYS':['sum', 'mean', 'std','max'],
                                                            
                                                            }).reset_index()

    gb_installment_by_curr.columns = ['_'.join(col).strip() for col in gb_installment_by_curr.columns.values]
    gb_installment_by_curr.rename({ 'SK_ID_CURR_': 'SK_ID_CURR'}, axis=1, inplace=True)

    gb_installment_short_term = gb_installment[gb_installment['NUM_INSTALMENT_NUMBER_max'] <= 12]
    gb_installment_long_term = gb_installment[gb_installment['NUM_INSTALMENT_NUMBER_max'] > 12]

    gb_installment_short_term_by_curr = gb_installment_short_term.groupby('SK_ID_CURR').agg({'COUNT_sum':'sum', 
                                                                                            'NUM_INSTALMENT_NUMBER_max':'mean', 
                                                                                            'MEAN_INSTALLMENT':'mean', 
                                                                                            'LATE_sum':'mean', 
                                                                                            'EARLY_sum':'mean', 
                                                                                            'DAYS_ENTRY_PAYMENT_max':'max', 
                                                                                            'AMT_PAYMENT_sum':'sum', 
                                                                                            
                                                                                            'LAST_90_DAYS_sum':'sum', 
                                                                                            'LAST_180_DAYS_sum':'sum', 
                                                                                            'LAST_365_DAYS_sum':'sum', 
                                                                                            'LAST_730_DAYS_sum':'sum',  
                                                                                            
                                                                                            'PAYMENT_LAST_90_DAYS_sum':'sum', 
                                                                                            'PAYMENT_LAST_180_DAYS_sum':'sum', 
                                                                                            'PAYMENT_LAST_365_DAYS_sum':'sum', 
                                                                                            'PAYMENT_LAST_730_DAYS_sum':'sum', 
                                                                                            'PAYMENT_LAST_730_DAYS_sum':'sum', 
                                                                                            
                                                                                            }).reset_index()

    # gb_installment_short_term_by_curr.columns = ['_'.join(col).strip() for col in gb_installment_short_term_by_curr.columns.values]
    # gb_installment_short_term_by_curr.rename(columns={'SK_ID_CURR_': 'SK_ID_CURR'}, inplace=True)

    gb_installment_short_term_by_curr.rename({ col: col + '_SHORT_TERM' for col in gb_installment_short_term_by_curr.columns if col != 'SK_ID_CURR' }, axis=1, inplace=True)

    gb_installment_long_term_by_curr = gb_installment_long_term.groupby('SK_ID_CURR').agg({'COUNT_sum':'sum', 
                                                                                            'NUM_INSTALMENT_NUMBER_max':'mean', 
                                                                                            'MEAN_INSTALLMENT':'mean', 
                                                                                            'LATE_sum':'mean', 
                                                                                            'EARLY_sum':'mean', 
                                                                                            'DAYS_ENTRY_PAYMENT_max':'max', 
                                                                                            'AMT_PAYMENT_sum':'sum', 
                                                                                            
                                                                                            'LAST_90_DAYS_sum':'sum', 
                                                                                            'LAST_180_DAYS_sum':'sum', 
                                                                                            'LAST_365_DAYS_sum':'sum', 
                                                                                            'LAST_730_DAYS_sum':'sum',  
                                                                                            
                                                                                            'PAYMENT_LAST_90_DAYS_sum':'sum', 
                                                                                            'PAYMENT_LAST_180_DAYS_sum':'sum', 
                                                                                            'PAYMENT_LAST_365_DAYS_sum':'sum', 
                                                                                            'PAYMENT_LAST_730_DAYS_sum':'sum', 
                                                                                            'PAYMENT_LAST_730_DAYS_sum':'sum', 
                                                                                                                                                                                    
                                                                                            }).reset_index()

    # gb_installment_long_term_by_curr.columns = ['_'.join(col).strip() for col in gb_installment_long_term_by_curr.columns.values]
    # gb_installment_long_term_by_curr.rename(columns={'SK_ID_CURR_': 'SK_ID_CURR'}, inplace=True)

    gb_installment_long_term_by_curr.rename({ col: col + '_LONG_TERM' for col in gb_installment_long_term_by_curr.columns if col != 'SK_ID_CURR' }, axis=1, inplace=True)

    gb_installment_by_curr = gb_installment_by_curr.merge(gb_installment_short_term_by_curr, on='SK_ID_CURR', how='left')
    gb_installment_by_curr = gb_installment_by_curr.merge(gb_installment_long_term_by_curr, on='SK_ID_CURR', how='left')
    return gb_installment_by_curr


# MERGING

In [15]:
appl_train = pd.read_csv(os.path.join(ROOT,'dseb63_application_train.csv'), index_col=0)

# ====== BUREAU ======

bureau = process_bureau()
# bureau = pd.read_parquet('../../data/dseb63_bureau_general_v3.parquet')
bureau_columns = bureau.columns

new_bureau_columns = {col: 'BUREAU_' + col for col in bureau_columns if col != 'SK_ID_CURR'} 
bureau.rename(columns=new_bureau_columns, inplace=True)

# ====== PREVIOUS APPLICATION ======

prev_app = process_app_prev()
prev_app_columns = prev_app.columns

new_prev_app_columns = {col: 'PREV_APP_' + col for col in prev_app_columns if col != 'SK_ID_CURR'}
prev_app.rename(columns=new_prev_app_columns, inplace=True)

# ====== INSTALLMENTS PAYMENTS ======

installments = process_installment_payment()
installments_columns = installments.columns

installments.drop(columns= [col for col in installments_columns if 'TARGET' in col], inplace=True)

new_installments_columns = {col: 'INSTALLMENTS_' + col for col in installments_columns if col != 'SK_ID_CURR'}
installments.rename(columns=new_installments_columns, inplace=True)

# ====== CREDIT CARD BALANCE ======

credit_card = process_credit_card_balance()
credit_card_columns = credit_card.columns

new_credit_card_columns = {col: 'CREDIT_CARD_' + col for col in credit_card_columns if col != 'SK_ID_CURR'}
credit_card.rename(columns=new_credit_card_columns, inplace=True)

# ====== POS CASH BALANCE ======

pos_cash = process_pos_cash_balance()
pos_cash_columns = pos_cash.columns

new_posh_cash_columns = {col: 'POS_CASH_' + col for col in pos_cash_columns if col != 'SK_ID_CURR'}
pos_cash.rename(columns=new_posh_cash_columns, inplace=True)

# ====== MERGE DATA ======

X = appl_train.merge(bureau, on='SK_ID_CURR', how='left')
X = X.merge(installments, on='SK_ID_CURR', how='left')
X = X.merge(pos_cash, on='SK_ID_CURR', how='left')
X = X.merge(credit_card, on='SK_ID_CURR', how='left')
X = X.merge(prev_app, on='SK_ID_CURR', how='left')


X_test_ = pd.read_csv(os.path.join(ROOT,'dseb63_application_test.csv'),  index_col=0)
X_test_ = X_test_.merge(bureau, on='SK_ID_CURR', how='left')
X_test_ = X_test_.merge(installments, on='SK_ID_CURR', how='left')
X_test_ = X_test_.merge(pos_cash, on='SK_ID_CURR', how='left')
X_test_ = X_test_.merge(credit_card, on='SK_ID_CURR', how='left')
X_test_ = X_test_.merge(prev_app, on='SK_ID_CURR', how='left')


In [16]:
def RELU(series):
    return series.apply(lambda x: max(0, x))

In [17]:
def process_df(df):
    # df = df.copy()
    
    df['EMERGENCYSTATE_MODE'].fillna('No', inplace=True)
    # df['FONDKAPREMONT_MODE'].fillna('Unknown', inplace=True)
    # df['WALLSMATERIAL_MODE'].fillna('Not Specified', inplace=True)
    df['OCCUPATION_TYPE'].replace('IT staff', 'High skill tech staff', inplace=True)
    df['OCCUPATION_TYPE'].replace('Realty agents', 'Sales staff', inplace=True)
    # df['OCCUPATION_TYPE'].replace('HR staff', 'Laborers', inplace=True)
    
    # 'Managers',
    df['OCCUPATION_TYPE'].replace(['High skill tech staff', 'HR staff', 'Secretaries'], np.nan, inplace=True)
    df['OCCUPATION_TYPE'].replace(['Cleaning staff', 'Cooking staff', 'Waiters/barmen staff', 'Private service staff', 'Low-skill Laborers', 'Security staff'], np.nan, inplace=True)


    
    df.drop(columns=['HOUSETYPE_MODE'], inplace=True)

    df['WALLSMATERIAL_MODE'].replace(['Block', 'Stone, brick'], 'Brick', inplace=True)
    df['WALLSMATERIAL_MODE'].replace(['Others', 'Mixed', 'Monolithic', 'Not Specified', 'Wooden'], np.nan, inplace=True)
    
    
    df['FONDKAPREMONT_MODE'].replace(['not specified', 'org spec account', 'reg oper account'], np.nan, inplace=True)


    map_week_day = {
    'MONDAY': 'week_day',
    'TUESDAY': 'week_day',
    'WEDNESDAY': 'week_day',
    'THURSDAY': 'week_day',
    'FRIDAY': 'week_day',
    'SATURDAY': 'weekend',
    'SUNDAY': 'weekend',
    }
    
    

    df['WEEKDAY_APPR_PROCESS_START'] = df['WEEKDAY_APPR_PROCESS_START'].map(map_week_day)

    map_edu = {
        'Lower secondary': 0,
        'Secondary / secondary special': 1,
        'Incomplete higher': 2,
        'Higher education': 3,
        'Academic degree': 5
    }

    df['NAME_EDUCATION_TYPE'] = df['NAME_EDUCATION_TYPE'].map(map_edu, na_action='ignore').astype(int).fillna(0)

    df['NAME_FAMILY_STATUS'].replace('Unknown', 'Single / not married', inplace=True)
    df['CODE_GENDER'].replace('XNA', 'F', inplace=True)
    
    df['DAYS_EMPLOYED'] = df['DAYS_EMPLOYED'].fillna(365243)
    # df['DAYS_EMPLOYED'] = df['DAYS_EMPLOYED'].replace(0, 365243)

    # others = df['NAME_INCOME_TYPE'].value_counts().index[4:]
    df['NAME_INCOME_TYPE'].replace(['Pensioner', 'Unemployed', 'Student','Businessman','Maternity leave'], np.nan, inplace=True)

    
    # df['NAME_TYPE_SUITE'].fillna('Unaccompanied', inplace=True)
    df['NAME_TYPE_SUITE'].replace(['Spouse, partner', 'Group of people', 'Other_A', 'Other_B'], np.nan, inplace=True)

    df['NAME_HOUSING_TYPE'].replace(['With parents', 'Municipal apartment', 'Rented apartment', 'Co-op apartment'], np.nan, inplace=True)

    df['OWN_CAR_AGE'].fillna(-100, inplace=True)
    df['TOTALAREA_MODE'].fillna(0, inplace=True)
    
    df['AGE_INT'] = -df['DAYS_BIRTH'] // 365
    # Steal code
    df['DAYS_LAST_PHONE_CHANGE'].replace(0, np.nan, inplace=True)
    
    def group_organizations(org_type):
        if not isinstance(org_type, str):
            return org_type
        if 'Trade' in org_type:
            return np.nan
        elif 'Industry' in org_type:
            return 'Industry'
        elif 'Business' in org_type:
            return 'Business Entity'
        elif 'Transport' in org_type:
            return 'Transport'
        elif 'University' in org_type:
            return 'School'
        elif org_type in  ['Unknown', 'Insurance','Services', 'Insurance', 'Restaurant', 'Housing', 'Hotel', 'Agriculture', 'Cleaning', 'Other', 'Advertising','Police', 'Electricity', 'Culture', 'Religion', 'Telecom', 'Emergency', 'Mobile', 'Postal', 'Legal Services', 'Security', 'Security Ministries', 'Government']:
            return np.nan
        else:
            return org_type
    
    df['ORGANIZATION_TYPE'].replace('XNA', 'Unknown', inplace=True)
    df['ORGANIZATION_TYPE'] = df['ORGANIZATION_TYPE'].apply(group_organizations)
    
    med_income = df.groupby(['ORGANIZATION_TYPE', 'NAME_EDUCATION_TYPE'])['AMT_INCOME_TOTAL'].transform('median')
    med_income2 = df.groupby('ORGANIZATION_TYPE')['AMT_INCOME_TOTAL'].transform('median')
    df['income_ratio'] = df['AMT_INCOME_TOTAL'] / med_income
    df['income_ratio2'] = df['AMT_INCOME_TOTAL'] / med_income2
    df['true_annuity_div_income'] = df['AMT_ANNUITY'] / med_income
    df['true_annuity_div_income2'] = df['AMT_ANNUITY'] / med_income2
    df['true_income_div_totalarea'] = med_income / df['TOTALAREA_MODE'].clip(0.001,1)
    df['true_income_div_totalarea2'] = med_income2 / df['TOTALAREA_MODE'].clip(0.001,1)
    
    df['annuity_income_percentage'] = df['AMT_ANNUITY'] / df['AMT_INCOME_TOTAL']
    df['car_to_birth_ratio'] = RELU(df['OWN_CAR_AGE'] / df['DAYS_BIRTH'])
    df['car_to_employ_ratio'] = RELU(df['OWN_CAR_AGE'] / df['DAYS_EMPLOYED'])
    df['children_ratio'] = df['CNT_CHILDREN'] / df['CNT_FAM_MEMBERS']
    df['credit_to_annuity_ratio'] = df['AMT_CREDIT'] / df['AMT_ANNUITY'] # check
    df['credit_to_goods_ratio'] = df['AMT_CREDIT'] / df['AMT_GOODS_PRICE']
    df['credit_to_income_ratio'] = df['AMT_CREDIT'] / df['AMT_INCOME_TOTAL']
    
    
    df['ext_sources_mean'] = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis=1)
    df['ext_sources_sum'] = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].sum(axis=1)
    df['ext_sources_var'] = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].var(axis=1)
    df['ext_sources_weighted'] = df.EXT_SOURCE_1 * 2 + df.EXT_SOURCE_2 * 3 + df.EXT_SOURCE_3 * 4
    df['EXT_SOURCE_MISSING_VALUES'] = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].isna().sum(axis=1)
    
    df['income_credit_percentage'] = df['AMT_INCOME_TOTAL'] / df['AMT_CREDIT']
    df['income_per_child'] = df['AMT_INCOME_TOTAL'] / (df['CNT_CHILDREN'] + 1e-5) * (df['CNT_CHILDREN'] > 0) 
    df['income_per_person'] = df['AMT_INCOME_TOTAL'] / df['CNT_FAM_MEMBERS']
    df['PAYMENT_RATE'] = df['AMT_ANNUITY'] / df['AMT_CREDIT']
    df['phone_to_birth_ratio'] = RELU(df['DAYS_LAST_PHONE_CHANGE'] / df['DAYS_BIRTH'])
    df['phone_to_employ_ratio'] = RELU(df['DAYS_LAST_PHONE_CHANGE'] / (df['DAYS_EMPLOYED'] + 1e-5)) * (df['DAYS_EMPLOYED'] != 0) 
    
    df['cnt_non_child'] = df['CNT_FAM_MEMBERS'] - df['CNT_CHILDREN']
    df['child_to_non_child_ratio'] = df['CNT_CHILDREN'] / df['cnt_non_child'] * (df['cnt_non_child'] > 0)
    df['income_per_non_child'] = df['AMT_INCOME_TOTAL'] / df['cnt_non_child']* (df['cnt_non_child'] > 0)
    df['credit_per_person'] = df['AMT_CREDIT'] / df['CNT_FAM_MEMBERS']* (df['cnt_non_child'] > 0)
    df['credit_per_child'] = df['AMT_CREDIT'] / (1 + df['CNT_CHILDREN'])
    df['credit_per_non_child'] = df['AMT_CREDIT'] / df['cnt_non_child']* (df['cnt_non_child'] > 0)

    df['ANNUITY_INCOME_PERC'] = df['AMT_ANNUITY'] / df['AMT_INCOME_TOTAL']
    df['DEBT_BURDEN_PER_WORKING_DAY'] = df['PAYMENT_RATE'] / (df['DAYS_EMPLOYED'] + 1e-5) * (df['DAYS_EMPLOYED'] != 0) 
    df['DEBT_BURDEN_PER_LIFE_DAY'] = df['PAYMENT_RATE'] / df['DAYS_BIRTH']
    df['CREDIT_GOODS_PRICE_RATIO1'] = (df['AMT_CREDIT'] - df['AMT_GOODS_PRICE']) /  df['AMT_GOODS_PRICE']
    df['CREDIT_GOODS_PRICE_RATIO2'] = (df['AMT_CREDIT'] - df['AMT_GOODS_PRICE']) /  df['AMT_CREDIT']
    df['CREDIT_DOWN_PAYMENT'] = df['AMT_GOODS_PRICE'] - df['AMT_CREDIT']

    df['sin_HOUR_APPR_PROCESS_START'] = np.sin(2 * np.pi * df['HOUR_APPR_PROCESS_START'] / 24)
    df['cos_HOUR_APPR_PROCESS_START'] = np.cos(2 * np.pi * df['HOUR_APPR_PROCESS_START'] / 24)
    df.drop(columns=['HOUR_APPR_PROCESS_START'], inplace=True)

    docs = [f for f in df.columns if 'FLAG_DOC' in f]
    df['NEW_DOC_IND_AVG'] = df[docs].mean(axis=1)
    df['NEW_DOC_IND_STD'] = df[docs].std(axis=1)
    df['NEW_DOC_IND_KURT'] = df[docs].kurtosis(axis=1)
    df['HAS_DOCUMENT'] = df[docs].max(axis=1)
    df['DOCUMENT_COUNT'] = df[docs].sum(axis=1)

    #Drop flag document  
    flag_document = [f for f in df.columns if 'FLAG_DOCUMENT_' in f]
    df.drop(columns=flag_document, inplace=True)
    
    df['LIVINGAREA_AVG'] = df['LIVINGAREA_AVG'].fillna(0)
    df['LANDAREA_AVG'] = df['LANDAREA_AVG'].fillna(0)
    df['FLOORSMAX_AVG'] = df['FLOORSMAX_AVG'].fillna(0)
    df['LIVINGAPARTMENTS_AVG'] = df['LIVINGAPARTMENTS_AVG'].fillna(0)
    df['YEARS_BUILD_AVG'] = df['YEARS_BUILD_AVG'].fillna(0)

    df['RATIO_AMT_GOODS_PRICE_TO_LIVINGAREA_AVG'] = (df['AMT_GOODS_PRICE'] / df['LIVINGAREA_AVG'].clip(0.05,1) +1e-5) * (df['LIVINGAREA_AVG'] / df['LIVINGAREA_AVG']+1e-5)
    df['RATIO_AMT_GOODS_PRICE_TO_LANDAREA_AVG'] = (df['AMT_GOODS_PRICE'] / df['LANDAREA_AVG'].clip(0.05,1) +1e-5) * (df['LANDAREA_AVG'] / df['LANDAREA_AVG']+1e-5)
    df['RATIO_AMT_GOODS_PRICE_TO_FLOORSMAX_AVG_AVG'] = (df['AMT_GOODS_PRICE'] / df['FLOORSMAX_AVG'].clip(0.05,1) +1e-5) * (df['FLOORSMAX_AVG'] / df['FLOORSMAX_AVG']+1e-5)
    df['RATIO_AMT_GOODS_PRICE_TO_LIVINGAPARTMENTS_AVG'] = (df['AMT_GOODS_PRICE'] / df['LIVINGAPARTMENTS_AVG'].clip(0.05,1) +1e-5) * (df['LIVINGAPARTMENTS_AVG'] / df['LIVINGAPARTMENTS_AVG']+1e-5)
    df['RATIO_AMT_GOODS_PRICE_TO_YEARS_BUILD_AVG'] = (df['AMT_GOODS_PRICE'] / df['YEARS_BUILD_AVG'].clip(0.05,1) +1e-5) * (df['YEARS_BUILD_AVG'] / df['YEARS_BUILD_AVG']+1e-5)
    
    # df['RATIO_AMT_GOODS_PRICE_TO_LIVINGAREA_AVG'] = df['AMT_GOODS_PRICE'] / df['LIVINGAREA_AVG'].clip(0.001,1)
    # df['RATIO_AMT_GOODS_PRICE_TO_LANDAREA_AVG'] = df['AMT_GOODS_PRICE'] / df['LANDAREA_AVG'].clip(0.001,1)
    # df['RATIO_AMT_GOODS_PRICE_TO_FLOORSMAX_AVG_AVG'] = df['AMT_GOODS_PRICE'] / df['FLOORSMAX_AVG'].clip(0.001,1)
    # df['RATIO_AMT_GOODS_PRICE_TO_LIVINGAPARTMENTS_AVG'] = df['AMT_GOODS_PRICE'] / df['LIVINGAPARTMENTS_AVG'].clip(0.001,1) 
    # df['RATIO_AMT_GOODS_PRICE_TO_YEARS_BUILD_AVG'] = df['AMT_GOODS_PRICE'] / df['YEARS_BUILD_AVG'].clip(0.001,1)
    
    df['RELIABILITY_IN_CUSTOMER_CITY'] = df['REG_CITY_NOT_LIVE_CITY'] + df['REG_CITY_NOT_WORK_CITY'] + df['REG_REGION_NOT_LIVE_REGION'] + df['REG_REGION_NOT_WORK_REGION'] + df['LIVE_CITY_NOT_WORK_CITY'] + df['LIVE_REGION_NOT_WORK_REGION']
    df['SUM_CONTACTS'] = df['FLAG_MOBIL'] + df['FLAG_EMP_PHONE'] + df['FLAG_WORK_PHONE'] + df['FLAG_CONT_MOBILE'] + df['FLAG_PHONE'] + df['FLAG_EMAIL']

    #Some features from bureau 
    df['TOTAL_ENQUIRIES_CREDIT_BUREAU'] = df[['AMT_REQ_CREDIT_BUREAU_DAY',
                                            'AMT_REQ_CREDIT_BUREAU_HOUR',
                                            'AMT_REQ_CREDIT_BUREAU_WEEK',
                                            'AMT_REQ_CREDIT_BUREAU_MON',
                                            'AMT_REQ_CREDIT_BUREAU_QRT',
                                            'AMT_REQ_CREDIT_BUREAU_YEAR']].sum(axis=1)
    
    
        
    # df['PCTG_ENQUIRIES_HOUR'] = df['AMT_REQ_CREDIT_BUREAU_HOUR'] / df['TOTAL_ENQUIRIES_CREDIT_BUREAU']
    # df['PCTG_ENQUIRIES_DAY'] = df['AMT_REQ_CREDIT_BUREAU_DAY'] / df['TOTAL_ENQUIRIES_CREDIT_BUREAU']
    df['PCTG_ENQUIRIES_WEEK'] = df['AMT_REQ_CREDIT_BUREAU_WEEK'] / df['TOTAL_ENQUIRIES_CREDIT_BUREAU']
    df['PCTG_ENQUIRIES_MON'] = df['AMT_REQ_CREDIT_BUREAU_MON'] / df['TOTAL_ENQUIRIES_CREDIT_BUREAU']
    df['PCTG_ENQUIRIES_QRT'] = df['AMT_REQ_CREDIT_BUREAU_QRT'] / df['TOTAL_ENQUIRIES_CREDIT_BUREAU']
    df['PCTG_ENQUIRIES_YEAR'] = df['AMT_REQ_CREDIT_BUREAU_YEAR'] / df['TOTAL_ENQUIRIES_CREDIT_BUREAU']

    df.drop(columns=['AMT_REQ_CREDIT_BUREAU_DAY','AMT_REQ_CREDIT_BUREAU_HOUR', 'AMT_REQ_CREDIT_BUREAU_WEEK'], inplace=True)

    missing_columns = ['APARTMENTS_AVG', 'BASEMENTAREA_AVG', 'YEARS_BEGINEXPLUATATION_AVG',
                   'YEARS_BUILD_AVG', 'COMMONAREA_AVG', 'ELEVATORS_AVG', 'ENTRANCES_AVG',
                   'FLOORSMAX_AVG', 'FLOORSMIN_AVG', 'LANDAREA_AVG', 'LIVINGAPARTMENTS_AVG',
                   'LIVINGAREA_AVG', 'NONLIVINGAPARTMENTS_AVG', 'NONLIVINGAREA_AVG']

    df['MISSING_GRADINGS'] = df[missing_columns].isna().sum(axis=1)
    # Reliability in customer city or region of residence
    df['RELIABILITY_IN_CUSTOMER_CITY'] = df['REG_CITY_NOT_LIVE_CITY'] + df['REG_CITY_NOT_WORK_CITY'] + df['REG_REGION_NOT_LIVE_REGION'] + df['REG_REGION_NOT_WORK_REGION'] + df['LIVE_CITY_NOT_WORK_CITY'] + df['LIVE_REGION_NOT_WORK_REGION']
    df['SUM_CONTACTS'] = df['FLAG_MOBIL'] + df['FLAG_EMP_PHONE'] + df['FLAG_WORK_PHONE'] + df['FLAG_CONT_MOBILE'] + df['FLAG_PHONE'] + df['FLAG_EMAIL']

    # numerical transformation
    df['DAYS_EMPLOYED'].replace(365243,0, inplace=True)
    df['days_employed_percentage'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']
    
    

    # vuxminhan params
    df['REGION_POPULATION_RELATIVE'] = np.sqrt(df['REGION_POPULATION_RELATIVE'])
    df['APARTMENTS_AVG'] = np.log1p(50 * df['APARTMENTS_AVG'])
    # df['YEARS_BEGINEXPLUATATION_AVG'] = df['YEARS_BEGINEXPLUATATION_AVG'] ** 30 # New
    df['YEARS_BUILD_AVG'] = df['YEARS_BUILD_AVG'] ** 3
    df['COMMONAREA_AVG'] = df['COMMONAREA_AVG'].clip(0.000001,1) ** (-1/200)
    df['ELEVATORS_AVG'] = df['ELEVATORS_AVG'] ** (1/40)
    df['ENTRANCES_AVG'] = df['ENTRANCES_AVG'] ** (1/3)
    df['FLOORSMAX_AVG'] = df['FLOORSMAX_AVG'] ** (1/2.5)
    df['FLOORSMIN_AVG'] = df['FLOORSMIN_AVG'] ** (1/2.2)
    df['LANDAREA_AVG'] = df['LANDAREA_AVG'] ** (1/5)
    df['LIVINGAPARTMENTS_AVG'] = df['LIVINGAPARTMENTS_AVG'] ** (1/3)
    df['LIVINGAREA_AVG'] = df['LIVINGAREA_AVG'] ** (1/3)
    df['NONLIVINGAPARTMENTS_AVG'] = df['NONLIVINGAPARTMENTS_AVG'] ** (1/7)
    df['NONLIVINGAREA_AVG'] = df['NONLIVINGAREA_AVG'] ** (1/5)
    df['OBS_30_CNT_SOCIAL_CIRCLE'] = df['OBS_30_CNT_SOCIAL_CIRCLE'] ** (1/7)
    df['DEF_30_CNT_SOCIAL_CIRCLE'] = df['DEF_30_CNT_SOCIAL_CIRCLE'] ** (1/7)
    df['OBS_60_CNT_SOCIAL_CIRCLE'] = df['OBS_60_CNT_SOCIAL_CIRCLE'] ** (1/7)
    df['DEF_60_CNT_SOCIAL_CIRCLE'] = df['DEF_60_CNT_SOCIAL_CIRCLE'] ** (1/7)

def process_external(df):
    df['ext_sources_mean'] = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis=1)
    df['ext_sources_sum'] = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].sum(axis=1)
    df['ext_sources_var'] = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].var(axis=1)
    df['ext_sources_weighted'] = df.EXT_SOURCE_1 * 2 + df.EXT_SOURCE_2 * 3 + df.EXT_SOURCE_3 * 4
   


In [18]:
process_df(X)
process_df(X_test_)
    
process_external(X)
process_external(X_test_)

In [19]:
X.head()

,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,RATIO_AMT_GOODS_PRICE_TO_YEARS_BUILD_AVG,RELIABILITY_IN_CUSTOMER_CITY,SUM_CONTACTS,TOTAL_ENQUIRIES_CREDIT_BUREAU,PCTG_ENQUIRIES_WEEK,PCTG_ENQUIRIES_MON,PCTG_ENQUIRIES_QRT,PCTG_ENQUIRIES_YEAR,MISSING_GRADINGS,days_employed_percentage
0,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,...,1.418984e+06,0,4,0.0,NaN,NaN,NaN,NaN,0,0.070862
1,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,297000.0,...,NaN,0,3,0.0,NaN,NaN,NaN,NaN,9,0.159905
2,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,513000.0,...,NaN,2,3,0.0,NaN,NaN,NaN,NaN,9,0.152418
3,0,Cash loans,M,N,Y,0,99000.0,490495.5,27517.5,454500.0,...,NaN,0,5,2.0,0.0,0.0,0.5,0.5,9,0.093737
4,0,Cash loans,M,Y,Y,0,360000.0,1530000.0,42075.0,1530000.0,...,NaN,2,4,0.0,NaN,NaN,NaN,NaN,9,0.023820


In [20]:
# If you want to free memory
del bureau, installments, pos_cash, credit_card, prev_app
gc.collect()

0

In [21]:
y = X['TARGET']
X.drop(columns=['TARGET', 'SK_ID_CURR'], inplace=True)

test_id = X_test_['SK_ID_CURR']
X_test_.drop(columns=['SK_ID_CURR'], inplace=True)


In [22]:
high_null_cols = [
    'BUREAU_SOLD_SUM_CREDIT',
    'BUREAU_BAD_DEBT_LAST_5_YEAR',
    'BUREAU_BAD_DEBT_FINISHED',
    'BUREAU_BAD_DEBT_SUM_CREDIT',
    'BUREAU_CLOSE_LATENCY_30_DAYS',
    'BUREAU_SUM_OTHER',
    'BUREAU_SUM_OVERDUE_MICROLOAN',
    'BUREAU_SUM_DEBT_MICROLOAN',
    'BUREAU_SUM_MICROLOAN',
    'PREV_APP_APPROVED_RATE_INTEREST_PRIMARY_MIN',
    'PREV_APP_APPROVED_RATE_INTEREST_PRIVILEGED_MAX',
    'PREV_APP_APPROVED_RATE_INTEREST_PRIVILEGED_MEAN',
    'PREV_APP_APPROVED_RATE_INTEREST_PRIMARY_MAX',
    'PREV_APP_APPROVED_RATE_INTEREST_PRIVILEGED_MIN',
    'PREV_APP_APPROVED_RATE_INTEREST_PRIMARY_MEAN',
    'BUREAU_SOLD_DURATION',
    'BUREAU_COUNT_SOLD',
    'BUREAU_SOLD_LAST_1000_DAYS',
    'BUREAU_SOLD_SUM_CREDIT', # 98.3

    'BUREAU_ACTIVE_SUM_CREDIT_30_DAYS', # 97.16
    'BUREAU_ACTIVE_SUM_DEBT_30_DAYS',
    'BUREAU_ACTIVE_SUM_OVERDUE_30_DAYS',

    'BUREAU_CLOSE_LATENCY_180_DAYS', # 96.6
    'BUREAU_CLOSE_SUM_DEBT_180_DAYS', # 96.54
    'BUREAU_CLOSE_SUM_OVERDUE_180_DAYS',
    'BUREAU_CLOSE_SUM_CREDIT_180_DAYS',
    # 'BUREAU_CLOSE_LATENCY_365_DAYS', #86

    'BUREAU_SUM_MORTGAGE', # 95.35 -> 0.5644
    'BUREAU_SUM_DEBT_MORTGAGE',
    'BUREAU_SUM_OVERDUE_MORTGAGE',

    'PREV_APP_REFUSED_AMT_DOWN_PAYMENT_STD', # 94.84

    'BUREAU_SUM_CREDIT_CAR_LOAN', # 93.611
    'BUREAU_SUM_OVERDUE_CAR_LOAN',
    'BUREAU_SUM_DEBT_CAR_LOAN',

    'BUREAU_CLOSE_SUM_CREDIT_LIFE_TIME', # 91.91

    'PREV_APP_REFUSED_AMT_ANNUITY_SKEW', # 91.5
    'PREV_APP_REFUSED_AMT_GOODS_PRICE_SKEW',
    'PREV_APP_REFUSED_AMT_CREDIT_SKEW', # 89.9
    'PREV_APP_REFUSED_AMT_APPLICATION_SKEW',

    'POS_CASH_LONG_TERM_LAST_36_MONTHS', # 88.5
    'POS_CASH_LONG_TERM_CNT_INSTALMENT',
    'POS_CASH_LONG_TERM_LONG_TERM',
    'POS_CASH_LONG_TERM_SK_DPD',
    'POS_CASH_LONG_TERM_SK_DPD_DEF',
    'POS_CASH_LONG_TERM_RATE_COMPLETED',
    'POS_CASH_LONG_TERM_SK_ID_PREV',
    'POS_CASH_LONG_TERM_NUM_INSTALMENT',
    'POS_CASH_LONG_TERM_LAST_12_MONTHS', # 0.56612

    'PREV_APP_REFUSED_RATE_DOWN_PAYMENT_MAX', # 85.3
    'PREV_APP_REFUSED_AMT_DOWN_PAYMENT_MAX',
    'PREV_APP_REFUSED_RATE_DOWN_PAYMENT_MEAN',
    'PREV_APP_REFUSED_RATE_DOWN_PAYMENT_MIN',
    'PREV_APP_REFUSED_AMT_DOWN_PAYMENT_MEAN',
    'PREV_APP_REFUSED_AMT_DOWN_PAYMENT_MIN',

    'PREV_APP_REFUSED_RATIO_GOODS_TO_ANNUITY_STD', # 84.7
    'PREV_APP_REFUSED_RATIO_APPLICATION_TO_ANNUITY_STD',
    'PREV_APP_REFUSED_CREDIT_ANNUITY_RATIO_STD',
    'PREV_APP_REFUSED_ANNUITY_PAYMENT_PRODUCT_STD',
    'PREV_APP_REFUSED_AMT_ANNUITY_STD',
    'PREV_APP_REFUSED_AMT_GOODS_PRICE_STD', # 0.56626

    'PREV_APP_REFUSED_APP_CREDIT_PERC_VAR', # 83.96
    'PREV_APP_REFUSED_AMT_CREDIT_STD',
    'PREV_APP_REFUSED_AMT_APPLICATION_STD',
    'CREDIT_CARD_PERCENTAGE_OF_MINIMUM_PAYMENTS_MISSED_sum', # 0.56654

    # 80

    # Add cc_balance_v2
    'CREDIT_CARD_PERCENTAGE_OF_MINIMUM_PAYMENTS_MISSED_var',
    'CREDIT_CARD_SUM_ALL_AMT_DRAWINGS_var',
    'CREDIT_CARD_RATIO_ALL_AMT_DRAWINGS_TO_ALL_CNT_DRAWINGS_var',
    'CREDIT_CARD_SUM_ALL_CNT_DRAWINGS_var',
    'CREDIT_CARD_CNT_DRAWINGS_ATM_CURRENT_std',
    'CREDIT_CARD_PERCENTAGE_OF_MINIMUM_PAYMENTS_MISSED_max',
    'CREDIT_CARD_SUM_ALL_AMT_DRAWINGS_max',
    'CREDIT_CARD_RATIO_ALL_AMT_DRAWINGS_TO_ALL_CNT_DRAWINGS_mean',
    'CREDIT_CARD_RATIO_ALL_AMT_DRAWINGS_TO_ALL_CNT_DRAWINGS_max',
    'CREDIT_CARD_RATIO_ALL_AMT_DRAWINGS_TO_ALL_CNT_DRAWINGS_min',



    'CREDIT_CARD_RATIO_ALL_AMT_DRAWINGS_TO_ALL_CNT_DRAWINGS_std' 
    'CREDIT_CARD_AMT_DRAWINGS_ATM_CURRENT_std',
    'CREDIT_CARD_AMT_DRAWINGS_POS_CURRENT_std',
    'CREDIT_CARD_SUM_ALL_CNT_DRAWINGS_mean',
    'CREDIT_CARD_AMT_DRAWINGS_ATM_CURRENT_mean',
    'CREDIT_CARD_CNT_DRAWINGS_ATM_CURRENT_mean',
    'CREDIT_CARD_AMT_DRAWINGS_ATM_CURRENT_max',
    'CREDIT_CARD_AMT_DRAWINGS_ATM_CURRENT_min',
    'CREDIT_CARD_SUM_ALL_AMT_DRAWINGS_mean',
    'CREDIT_CARD_AMT_DRAWINGS_POS_CURRENT_min',
    'CREDIT_CARD_AMT_DRAWINGS_POS_CURRENT_max',
    'CREDIT_CARD_AMT_DRAWINGS_POS_CURRENT_mean',
    'CREDIT_CARD_RATIO_ALL_AMT_DRAWINGS_TO_ALL_CNT_DRAWINGS_mean', # 0.56664

    'BUREAU_GENERAL_AMT_ANNUITY_std',
    'BUREAU_ACTIVE_SUM_OVERDUE_LIFE_TIME',
    'BUREAU_ACTIVE_SUM_CREDIT_LIFE_TIME', # 0.56670

]

In [23]:
for col in high_null_cols:
    if col in X_test_.columns:
        X_test_[col].fillna(0, inplace=True)
        X[col].fillna(0,inplace= True)

In [24]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PowerTransformer, OrdinalEncoder, LabelEncoder, MinMaxScaler

In [25]:
ordinal_cols = ['FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'EMERGENCYSTATE_MODE', 'CODE_GENDER', 'NAME_CONTRACT_TYPE']
float_cols = []
int_cols = []
cate_cols = []
flag_cols = []
for col in X.columns:
    if col not in ordinal_cols:
        if X[col].dtype == 'float64':
            float_cols.append(col)
            X[col] = X[col].astype('float64')
        elif X[col].dtype == 'int64':

            int_cols.append(col)
            X[col] = X[col].astype('int64')
        elif X[col].dtype == 'bool':
            int_cols.append(col)
            X[col] = X[col].astype('int64')

        else:
            cate_cols.append(col)
            X[col] = X[col].astype('str')
    else:
        X[col] = X[col].astype('str')

In [26]:
# Test std to var on ratio
for col in X.columns:
    if '_std' in col.lower():
        # if '_ratio' in col.lower():
        if X[col].max() < 50_000: # Avoid large number
            X[col] = X[col]**2
            X_test_[col] = X_test_[col]**2
            print(col)

BUREAU_GENERAL_CREDIT_DURATION_std
BUREAU_GENERAL_DEBT_PERCENTAGE_std
BUREAU_GENERAL_UTILIZATION_RATIO_std
INSTALLMENTS_DIFF_std
INSTALLMENTS_LATE_std
INSTALLMENTS_EARLY_std
INSTALLMENTS_LATENESS_LAST_180_DAYS_std
INSTALLMENTS_LATENESS_LAST_730_DAYS_std
INSTALLMENTS_EARLYNESS_LAST_180_DAYS_std
INSTALLMENTS_EARLYNESS_LAST_730_DAYS_std
INSTALLMENTS_PAYMENT_LATENESS_LAST_365_DAYS_std
INSTALLMENTS_PAYMENT_LATENESS_LAST_730_DAYS_std
INSTALLMENTS_PAYMENT_EARLYNESS_LAST_365_DAYS_std
INSTALLMENTS_PAYMENT_EARLYNESS_LAST_730_DAYS_std
CREDIT_CARD_RATIO_ALL_AMT_DRAWINGS_TO_ALL_CNT_DRAWINGS_std
CREDIT_CARD_6_MONTH_RATE_OF_PAYBACK_std
CREDIT_CARD_6_MONTH_AMT_INST_MIN_REGULARITY_std
CREDIT_CARD_6_MONTH_PRINCIPIAL_RATE_std
CREDIT_CARD_12_MONTH_RATE_OF_PAYBACK_std
CREDIT_CARD_12_MONTH_AMT_INST_MIN_REGULARITY_std
CREDIT_CARD_12_MONTH_PRINCIPIAL_RATE_std
CREDIT_CARD_36_MONTH_AMT_INST_MIN_REGULARITY_std
PREV_APP_PREV_CREDIT_ANNUITY_RATIO_STD
PREV_APP_PREV_DOWN_PAYMENT_RATIO_STD
PREV_APP_PREV_GOODS_PRICE_R

In [27]:
for col in float_cols:
    X[col].fillna(X[col].mean(), inplace=True)
    X_test_[col].fillna(X[col].mean(), inplace=True)
    
for col in int_cols:
    X[col].fillna(X[col].mean(), inplace=True)
    X_test_[col].fillna(X[col].mean(), inplace=True)
    
for col in cate_cols:
    X[col].fillna(np.nan, inplace=True)
    X_test_[col].fillna(np.nan, inplace=True)

In [28]:
from tqdm import tqdm
def analyze_feat(series):
    
    # Check the type of the feature
    type_ = series.dtype
    if type_ != 'object':
        
    
        # Check the skewness of the feature
        skew = series.skew()
        
        # Check the kurtosis of the feature
        kurt = series.kurt()
        
        # Check the missing values of the feature
        missing = series.isnull().sum()
        
        # Can convert to interger?
        is_interger = 0
        
        # Min, max, mean, median, std, unique values
        min_ = series.min()
        max_ = series.max()
        mean_ = series.mean()
        median_ = series.median()
        std_ = series.std()
    else:
        skew = 0
        kurt = 0
        missing = 0
        is_interger = False
        min_ = 0
        max_ = 0
        mean_ = 0
        median_ = 0
        std_ = 0   
        
    unique_ = series.nunique()
    
    # Return the results as a row 
    return (type_, skew, kurt, missing, is_interger, min_, max_, mean_, median_, std_, unique_)

def analyze_column(feature, df):
    # Get the results for a specific feature
    return [feature] + list(analyze_feat(df[feature]))
   
def analyze_df(df):
    features = df.columns
    if 'TARGET' in features:
        df.drop(columns=['TARGET'], inplace=True)
        features = df.columns
    # Create a DataFrame to store the results
    
    results = []
    # Loop through each feature
    for feature in tqdm(features):
        # Get the results
        result = analyze_feat(df[feature])
        
        # Append the results as a new row
        results.append([feature] + list(result))
    
    return pd.DataFrame(results, columns=['Feature', 'Type', 'Skewness', 'Kurtosis', 'Missing', 'Is Integer', 'Min', 'Max', 'Mean', 'Median', 'Std', 'Unique'])
    
    
def power_scaler_col(df, skewness = 8, kurtosis = 50, use_cache = True):
    # Return which columns for power transformation, which for standard scaler

    df_num = df.select_dtypes(include=[np.number])
    df_num.fillna(0, inplace=True)
    df_result = analyze_df(df)

    # Nunique < 100 -> Categorical feature -> Scale
    df_result['Mask_nunique'] = df_result['Unique'] > 100
    
    # Skewness > 10 -> Power transformation
    df_result['Mask_skew'] = df_result['Skewness'].abs() > skewness
    
    # Kurtosis > 15 -> Power transformation
    df_result['Mask_kurt'] = (df_result['Kurtosis']-3).abs() > kurtosis
    
    # Missing > 9/10 dataset -> Scale
    df_result['Mask_missing'] = df_result['Missing'] < 0.5 * df.shape[0]
    
    df_result['Mask'] = df_result['Mask_nunique'] & (df_result['Mask_skew'] | df_result['Mask_kurt']) & df_result['Mask_missing']
    
    df_result['Min_Max'] = ( df_result['Min'] == 0) & (df_result['Max'] < 20) & (df_result['Unique'] < 50) # 50/50

    power_col = df_result[df_result['Mask'] == 1].Feature.tolist()
    standard_col = df_result[(df_result['Mask'] == 0) & (df_result['Min_Max'] == 0)].Feature.tolist()
    min_max_col = df_result[(df_result['Mask'] == 0) & (df_result['Min_Max'] == 1)].Feature.tolist()
    
    return power_col, standard_col, min_max_col

In [29]:
power_col, standard_col, min_max_col = power_scaler_col(X[float_cols+int_cols], skewness= 3, kurtosis = 20, use_cache=False)
len(power_col), len(standard_col), len(min_max_col)

100%|██████████| 869/869 [00:04<00:00, 184.06it/s]


(468, 333, 68)

In [30]:
X_columns = X.columns
y = y.to_numpy()

KNNIMPUTE is too long. Use KNN instead

In [31]:
knn_imputer = Pipeline([
    ('scaler', StandardScaler()),
    ('knn', KNeighborsClassifier(n_neighbors=500, n_jobs=-1))  
])

knn_imputer_2 = Pipeline([
    ('scaler', StandardScaler()),
    ('knn', KNeighborsClassifier(n_neighbors=500, n_jobs=-1))  
])

knn_imputer_3 = Pipeline([
    ('scaler', StandardScaler()),
    ('knn', KNeighborsClassifier(n_neighbors=500, n_jobs=-1))  
])

onehot_transformer = OneHotEncoder(handle_unknown='ignore')
power_transformer = PowerTransformer()
ordinal_transformer = OrdinalEncoder()
scaler_transformer = StandardScaler()
min_max_transformer = MinMaxScaler()

preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', onehot_transformer, cate_cols),
        ('power', power_transformer, power_col), # power_transformer
        ('ordinal', ordinal_transformer, ordinal_cols),
        ('scale', scaler_transformer, standard_col),
        ('min_max', min_max_transformer, min_max_col)
        
    ],
    n_jobs=-1
)


In [32]:
weak_df_col = ['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'credit_to_annuity_ratio', 'annuity_income_percentage']
weak_df_col_2 = ['DAYS_ID_PUBLISH', 'DAYS_REGISTRATION', 'days_employed_percentage', 'car_to_birth_ratio', 'NAME_EDUCATION_TYPE', 'AGE_INT']
weak_df_col_3 = ['PAYMENT_RATE',  'PREV_APP_APPROVED_CREDIT_DOWNPAYMENT_SUM', 'POS_CASH_GENERAL_CNT_INSTALMENT_mean', 'POS_CASH_GENERAL_CNT_INSTALMENT_FUTURE_sum', 'AMT_ANNUITY']


In [33]:
from sklearn.utils import resample
from sklearn.linear_model import LogisticRegression

def log_resample_data(X_train, y_train, rate = 0.2, decay = 0.7, return_idx = False, sampling_strategy = 0.1, modelling = False):
    
    posidx = (y_train == 1)
    negidx = (y_train == 0)

    # X_train = np.concatenate([X_train, y_train2], axis=1)
    posidx = np.where(posidx)[0]
    negidx = np.where(negidx)[0]

    n_samples = int(len(negidx) * rate)

    if posidx.sum() < n_samples: # Not enough positive samples
        return X_train, y_train
    
    if modelling:
        if sampling_strategy == 'auto':
            sampling_strategy = 0.5 * (len(posidx) / len(negidx))

        start = time.time()
        lr = LogisticRegression(max_iter=5000, C=0.001, tol= 1e-5, random_state=42, class_weight='balanced') # Train a weak Log model
        lr.fit(X_train, y_train)
        y_temp = lr.predict_proba(X_train[posidx, :])[:,1] # Get the probability of being positive
        end = time.time()
        print('Log fitted', end-start)


        border_idx = np.where(y_temp < sampling_strategy)[0] # Find the border samples ()
        mask = np.zeros(len(y_temp), dtype=bool) 
        mask[border_idx] = True

        important_idx = posidx[mask]
        not_important_idx = posidx[~mask]

        len_import = len(important_idx)
        len_not_import = len(not_important_idx)
        len_data = len(posidx)

        reverse_decay = (len_data - len_not_import * decay) / len_import

        upscale_important = int(len_import/len_data * n_samples * reverse_decay)
        upscale_not_important = int(len_not_import/len_data * n_samples * decay)

        important_idx2 = resample(important_idx, n_samples=upscale_important, random_state=42)
        not_important_idx2 = resample(not_important_idx, n_samples=upscale_not_important, random_state=42)

        print('Near border sample:',len_import, 'Up to', upscale_important)
        print('Far border sample:',len_not_import, 'Up to', upscale_not_important)

        posidx2 = np.concatenate([important_idx2, not_important_idx2])
        

    else:
        posidx2 = resample(posidx, n_samples=n_samples, random_state=42)

    idx = np.concatenate([posidx2, negidx])
    np.random.shuffle(idx)
    
    if return_idx:
        return idx

    if isinstance(X_train, pd.DataFrame):
        return X_train.iloc[idx,:], y_train[idx]
    else:
        return X_train[idx,:], y_train[idx]


In [34]:
X_train = X.copy()

In [35]:
print('KNN-ing')
start = time.time()
knn_imputer.fit(X_train[weak_df_col], y) # fit on original data
knn_imputer_2.fit(X_train[weak_df_col_2], y) # fit on original data
knn_imputer_3.fit(X_train[weak_df_col_3], y)


weak_feature_train = knn_imputer.predict_proba(X_train[weak_df_col])[:,1]
weak_feature_val = knn_imputer.predict_proba(X_test_[weak_df_col])[:,1]
print('1')

weak_feature_train_2 = knn_imputer_2.predict_proba(X_train[weak_df_col_2])[:,1]
weak_feature_val_2 = knn_imputer_2.predict_proba(X_test_[weak_df_col_2])[:,1]
print('2')

weak_feature_train_3 = knn_imputer_3.predict_proba(X_train[weak_df_col_3])[:,1]
weak_feature_val_3 = knn_imputer_3.predict_proba(X_test_[weak_df_col_3])[:,1]
print('3')

end = time.time()
print('Weak features fitted', end-start)

print('Transforming data')
start = time.time()

preprocessor.fit(X) # fit on original data
X_train = preprocessor.transform(X_train)
X_test = preprocessor.transform(X_test_)
end = time.time()
print('Data transformed', end-start)


X_train = np.concatenate([X_train, 
                            weak_feature_train.reshape(-1,1), 
                            weak_feature_train_2.reshape(-1,1),
                            weak_feature_train_3.reshape(-1,1)
                            ], axis=1)
X_test = np.concatenate([X_test, 
                        weak_feature_val.reshape(-1,1), 
                        weak_feature_val_2.reshape(-1,1),
                        weak_feature_val_3.reshape(-1,1)
                        ], axis=1)

cat_feature = preprocessor.named_transformers_['onehot'].get_feature_names_out(cate_cols)
feature_names = np.concatenate([cat_feature, 
                                power_col, 
                                ordinal_cols, 
                                standard_col, 
                                min_max_col, 
                                ['a', 'b','c']])

KNN-ing
1
2
3
Weak features fitted 36.57365679740906
Transforming data
Data transformed 81.44982743263245


In [36]:
idx = log_resample_data(X_train, y, rate = UPSAMPLE_RATIO, decay = 1.05, return_idx = True, sampling_strategy = 0.63)
X_resampled = X_train[idx,:]
y_resampled = y[idx]

In [37]:
X_resampled = pd.DataFrame(X_resampled, columns=feature_names)
X_test = pd.DataFrame(X_test, columns=feature_names)

for col in feature_names:
    if '_nan' in col:
        print(col)
        X_resampled.drop(columns=[col], inplace=True)
        X_test.drop(columns=[col], inplace=True)

NAME_TYPE_SUITE_nan
NAME_INCOME_TYPE_nan
NAME_HOUSING_TYPE_nan
OCCUPATION_TYPE_nan
ORGANIZATION_TYPE_nan
FONDKAPREMONT_MODE_nan
WALLSMATERIAL_MODE_nan


In [38]:
bad_flag = ['FLAG_MOBIL',
 'FLAG_EMP_PHONE' ,'FLAG_CONT_MOBILE', 'FLAG_EMAIL',
 'REG_REGION_NOT_LIVE_REGION', 'REG_REGION_NOT_WORK_REGION',
 'LIVE_REGION_NOT_WORK_REGION', 'REG_CITY_NOT_WORK_CITY',
 'LIVE_CITY_NOT_WORK_CITY', 'EMERGENCYSTATE_MODE']

X_resampled.drop(columns=bad_flag, inplace=True)
X_test.drop(columns=bad_flag, inplace=True)

In [39]:
features = X_resampled.columns

In [40]:
del X 
del X_train 
del X_test_
gc.collect()

52

# Modelling

In [41]:
from sklearn.metrics import make_scorer, roc_auc_score
def gini_coefficient(y_true, y_pred):
    """
    Calculate the Gini coefficient using predictions and true labels.
    
    Parameters:
    y_true (array-like): True binary labels.
    y_pred (array-like): Predicted probabilities.
    
    Returns:
    float: Gini coefficient.
    """
    auc = roc_auc_score(y_true, y_pred)  # AUC calculation
    return 2 * auc - 1  # Gini coefficient
gini_scorer = make_scorer(gini_coefficient, needs_proba=True)

In [42]:
logistic_model = LogisticRegression(random_state=42, max_iter=10000, class_weight='balanced', C = 0.01, tol = 1e-5, n_jobs=-1)

In [43]:
start = time.time()
logistic_model.fit(X_resampled[features], y_resampled)
y_pred = logistic_model.predict_proba(X_test[features])
end = time.time()
print('Fitting time:', end-start)

Fitting time: 220.116149187088


In [44]:
submission = pd.DataFrame({
    'SK_ID_CURR': test_id,
    'TARGET': y_pred[:, 1]
})


In [45]:
submission.to_csv('submission.csv', index = False)